## Konfiguracja

In [ ]:
from google.colab import userdata

In [ ]:
%cd /
from google.colab import drive
drive.mount('/content/gdrive/')

!ln -s /content/gdrive/My\ Drive/ ./mydrive

%cd /content/gdrive/MyDrive/projekt_inzynierski

/
Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
ln: failed to create symbolic link './mydrive/My Drive': File exists
/content/gdrive/MyDrive/projekt_inzynierski


In [ ]:
!pip install sentence-transformers -q
!pip install -q faiss-cpu sentence-transformers
!python -m spacy download en_core_web_sm
!python -m spacy download pl_core_news_sm
!pip install gliner
!python -m spacy download en_core_web_md
!python -m spacy download pl_core_news_lg
!pip install spacy gliner transformers torch
!pip install langdetect!pip install sentence-transformers -q
!pip install -q faiss-cpu sentence-transformers
!python -m spacy download en_core_web_sm
!python -m spacy download pl_core_news_sm
!pip install gliner
!python -m spacy download en_core_web_md
!python -m spacy download pl_core_news_lg
!pip install spacy gliner transformers torch
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 88.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 5.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 33.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('e

In [ ]:
import faiss
import torch
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from math import log
from numpy.linalg import norm
import re
from gliner import GLiNER
import spacy
from spacy.matcher import Matcher
from spacy.util import filter_spans
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn.functional as F
from langdetect import detect, DetectorFactory
import difflib

Modele

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_e5 = SentenceTransformer('intfloat/multilingual-e5-base', device=device)
model_gliner = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
nlp_pl = spacy.load("pl_core_news_lg")
nlp_en = spacy.load("en_core_web_md")device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_e5 = SentenceTransformer('intfloat/multilingual-e5-base', device=device)
model_gliner = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
nlp_pl = spacy.load("pl_core_news_lg")
nlp_en = spacy.load("en_core_web_md")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Dataframy

In [ ]:
df_songs = pd.read_parquet("../df_full_with_embeddings.parquet")

## Data preparation dodatkowe

zamień tag_list na listę

In [ ]:
def _ensure_list(x):
    if isinstance(x, list): return x
    if pd.isna(x) or x is None: return []
    return [t.strip() for t in str(x).split(",") if t.strip()]

df_songs["tags_list"] = df_songs["tags"].apply(_ensure_list)
df_songs["tag_count"] = df_songs["tags_list"].apply(len)
df_songs.head(5)

Uzupełnij tagi ich tagami nadrzędnymi dla lepszego dopasowania (to trzeba przenieść do innego pliku i poprawić bazę)

In [ ]:
ALSO_ADD_PARENT = {
    # rock
    "progressive rock": "rock",
    "classic rock": "rock",
    "indie rock": "rock",
    "hard rock": "rock",
    "pop rock": "rock",
    "psychedelic rock": "rock",
    "punk rock": "rock",
    "blues rock": "rock",
    "post rock": "rock",

    # metal
    "heavy metal": "metal",
    "death metal": "metal",
    "black metal": "metal",
    "doom metal": "metal",
    "thrash metal": "metal",
    "melodic death metal": "metal",
    "symphonic metal": "metal",
    "gothic metal": "metal",
    "nu metal": "metal",
    "progressive metal": "metal",
    "power metal": "metal",
    "metalcore": "metal",

    # pop / electronic
    "indie pop": "pop",
    "synthpop": "pop",
    "drum and bass": "electronic",
}

PARENT_KEYWORDS = {
    "rock": ["rock"],
    "metal": ["metal"],
    "pop": ["pop", "britpop"],
    "hip hop": ["hip hop", "rap"],
    "electronic": ["electronic", "techno", "house", "trance", "idm", "downtempo", "electro", "ambient"],
    "jazz": ["jazz"],
    "classical": ["classical"],
    "punk": ["punk"],
    "folk": ["folk"],
    "blues": ["blues"],
    "country": ["country"],
    "reggae": ["reggae"],
}

def expand_tags(tag_list):
    if tag_list is None or (isinstance(tag_list, float) and pd.isna(tag_list)):
        tag_list = []

    tags = {str(t).lower().replace("_", " ").strip() for t in tag_list}

    for child, parent in ALSO_ADD_PARENT.items():
        if child in tags:
            tags.add(parent)

    for parent, kws in PARENT_KEYWORDS.items():
        if any(kw in t for t in tags for kw in kws):
            tags.add(parent)

    return sorted(tags)

df_songs["tags_list"] = df_songs["tags_list"].apply(expand_tags)


In [ ]:
df_songs["tags_list"] = df_songs["tags_list"].apply(
    lambda lst: [tag.replace("_", " ") for tag in lst]
)

# Algorytm dopasowania utworów

## Przygotowanie

Budowanie inverted index do późniejszego scorowania utworów

In [ ]:
df_songs = df_songs.reset_index(drop=True)

def build_inverted_index(df_songs: pd.DataFrame, tags_col: str = "tags_list"):
    inv = defaultdict(list)
    df_count = defaultdict(int)

    for i, tags in enumerate(df_songs[tags_col]):
        if not tags:
            continue
        seen = set()
        for t in tags:
            inv[t].append(i)
            if t not in seen:
                df_count[t] += 1
                seen.add(t)

    for t in inv:
        inv[t] = np.asarray(inv[t], dtype=np.int32)

    return inv, df_count

INV_INDEX, DF_COUNT = build_inverted_index(df_songs, tags_col="tags_list")
N_SONGS = len(df_songs)
AVG_TAG_LEN = float(df_songs["tag_count"].mean()) if "tag_count" in df_songs else 10.0

Config do wszystkich parametrów algorytmu

In [ ]:
# EKSTRAKCJA I DOPASOWANIE (NLP / E5 / GLiNER)
EXTRACTION_CONFIG = {
    "gliner_threshold": 0.15,
    "tag_similarity_threshold": 0.82,
    "tag_similarity_threshold_lenient": 0.75,
    "audio_confidence_threshold": 0.78,
    "fuzzy_cutoff": 0.9
}

# SCORING
SCORING_CONFIG = {
    "use_idf": False,
    "query_pow": 1.0,
    "audio_weight": 0.4,
}

# 3. POBIERANIE KANDYDATÓW - RETRIEVAL
RETRIEVAL_CONFIG = {
    "n_candidates": 400,
    "flat_delta": 0.05,
}

# 4. TIERY JAKOŚCIOWE I ZBIÓR ROBOCZY
WORKSET_CONFIG = {
    "min_absolute_high": 0.75,
    "min_absolute_mid": 0.50,
    "target_pool_size": 100,
    "min_required_size": 15,
    "popularity_rescue_ratio": 0.2,
}

# 5. SEGMENTACJA POPULARNOŚCI
POPULARITY_CONFIG = {
    "p_high": 70,
    "p_mid": 35,
    "mix": {
        "high": 0.40,
        "mid":  0.35,
        "low":  0.25,
    },
    "forced_popular": 2,
    "forced_popular_min": 80,
}

# 6. FINALNY SAMPLING
SAMPLING_CONFIG = {
    "final_n": 15,
    "alpha": 2.0,
    "shuffle": True,
}

## Wyciąganie relevant phrases z prompta

In [ ]:
DetectorFactory.seed = 0

In [ ]:
# parsowanie gramatyczne, Matcher
nlp_pl = spacy.load("pl_core_news_lg")
nlp_en = spacy.load("en_core_web_md")

In [ ]:
GENERIC_LEMMAS = [
    "muzyka", "utwór", "piosenka", "kawałek", "playlista", "lista", "numer", "rok", "klimat", "styl",
    "music", "song", "track", "playlist", "list", "number", "vibe", "tune", "genre", "style"
]

GENERIC_VERBS = [
    "szukać", "poszukiwać", "chcieć", "pragnąć", "potrzebować", "woleć", "wymagać",
    "być", "mieć", "znajdować", "znaleźć", "słuchać", "posłuchać", "grać", "zależeć",
    "słuchać", "posłuchać", "usłyszeć", "grać", "zagrać", "puszczać", "puścić", "odtworzyć", "zapodać",
    "prosić", "polecić", "polecać", "rekomendować", "sugerować", "zaproponować", "dawać", "dać",

    "search", "look", "find", "want", "need", "desire", "wish", "require",
    "be", "have", "get",
    "listen", "hear", "play", "replay", "stream",
    "give", "recommend", "suggest", "show", "provide",
]

NEGATION_TERMS = [
    "nie", "bez", "mało", "zero", "ani", "żaden", "brak", "mniej",
    "no", "not", "without", "less", "non", "neither", "nor", "lack", "zero"
]

GENRE_MODIFIERS = [
    "post", "hard", "heavy", "soft", "pop", "alt", "alternative", "indie",
    "nu", "neo", "progressive", "psychedelic", "acid", "experimental",
    "symphonic", "death", "black", "thrash", "doom", "power", "speed",
    "glam", "gothic", "tech", "deep", "classic", "modern", "electronic",
    "industrial", "melodic", "folk", "lo-fi", "lofi", "ambient", "synth",
    "electro", "acoustic", "instrumental", "gangsta", "brit", "new", "dark",

    "alternatywny", "progresywny", "psychodeliczny", "symfoniczny",
    "ciężki", "klasyczny", "elektroniczny", "gotycki", "melodyjny",
    "eksperymentalny", "współczesny", "industrialny", "akustyczny",
    "instrumentalny", "polski", "zagraniczny"
]

In [ ]:
def create_matcher_for_nlp(nlp_instance):
    """Tworzy obiekt Matcher przypisany do konkretnego modelu językowego"""
    matcher = Matcher(nlp_instance.vocab)

    # Warunek wykluczający - rzeczownik ktorego nie ma na liście wykluczającej
    noun_filter = {
        "POS": {"IN": ["NOUN", "PROPN"]},
        "IS_STOP": False,
        "LEMMA": {"NOT_IN": GENERIC_LEMMAS}
    }

    matcher.add("FRAZA", [
        # Samodzielny rzeczownik lub nazwa własna
        [noun_filter],

        # Przymiotnik + Rzeczownik (hyper pop, progressive rock etc.)
        # [
        #     {"POS": "ADJ", "LEMMA": {"IN": GENRE_MODIFIERS}},
        #     noun_filter
        # ],
        [
            {"POS": "ADJ", "LEMMA": {"NOT_IN": GENERIC_LEMMAS}},
            noun_filter
        ],

        # Wyrażenie przyimkowe (opcjonalnie poprzedzone przysłówkiem) (do tańca, z klimatem, prosto z serca)
        [{"POS": "ADV", "OP": "?"}, {"POS": "ADP"}, noun_filter],

        # Przysłówek + Przymiotnik (bardzo wesoła, niezwykle głośny)
        [{"POS": "ADV"}, {"POS": "ADJ", "IS_STOP": False}],

        # Samodzielny przymiotnik ("rockowa", "spokojne")
        [{"POS": "ADJ", "IS_STOP": False}],

        # Rzeczowniki złożone (Dwa rzeczowniki obok siebie) (post rock", "hip hop", "death metal")
        [noun_filter, noun_filter],

        # Rozbudowana fraza przymiotnikowa z przyimkiem ("dobra do tańca", "idealny na imprezę")
        [{"POS": "ADV", "OP": "?"}, {"POS": "ADJ"}, {"POS": "ADP"}, noun_filter],

        # Fraza czasownikowa (Czasownik + Dopełnienie)
        [
            {"POS": "VERB", "LEMMA": {"NOT_IN": GENERIC_VERBS}},
            {"POS": {"IN": ["NOUN", "PROPN", "ADJ"]}, "IS_STOP": False, "LEMMA": {"NOT_IN": GENERIC_LEMMAS}, "OP": "+"}
        ],

        # liczby, (np. "z lat 90")
        [{"POS": "ADP"}, {"POS": "NOUN"}, {"IS_DIGIT": True}],
        [{"POS": "NOUN"}, {"IS_DIGIT": True}],

        # Gatunki z myślnikiem
        [{"POS": {"IN": ["NOUN", "PROPN", "ADJ"]}}, {"ORTH": "-"}, {"POS": {"IN": ["NOUN", "PROPN", "ADJ"]}}],
    ])
    return matcher

# tworzenie matcherów (raz)
matcher_pl = create_matcher_for_nlp(nlp_pl)
matcher_en = create_matcher_for_nlp(nlp_en)


In [ ]:
def is_span_negated(doc, start_index, window=2):
    """
    Sprawdza, czy przed frazą stoi słowo przeczące.
    Patrzy 'window' tokenów wstecz.
    """
    lookback = max(0, start_index - window)
    preceding_tokens = doc[lookback:start_index]

    for token in preceding_tokens:
        if token.text.lower() in NEGATION_TERMS:
            return True
    return False

In [ ]:
def extract_relevant_phrases(prompt, current_nlp, current_matcher):
    # Setup językowy
    prompt = prompt.lower()
    prompt_clean = prompt.strip()

    if not prompt_clean:
        return []

    doc = current_nlp(prompt)
    matcher_matches = current_matcher(doc)

    # Konwersja matchy na obiekty Span
    matcher_spans = [doc[start:end] for match_id, start, end in matcher_matches]
    combined_spans = filter_spans(matcher_spans) # usuwa nakładające się frazy, zostawia najdłuższe dopasowanie

    final_phrases = []

    for span in combined_spans:
        if is_span_negated(doc, span.start):
            continue
        final_phrases.append(span.text.lower())

    unique_phrases = list(set([p.strip() for p in final_phrases if len(p.strip()) > 2]))

    print(f"Prompt: '{prompt}' \n-> {unique_phrases}")

    return unique_phrases

In [ ]:
TEST_PROMPT_1 = "szukam muzyki rockowej, ale takiej pełnej spokoju i bardzo wesołej, trochę do tańca"
TEST_PROMPT_2 = "rock, pop, dance i coś do tańca, zależy mi na maksymalnej energii"
TEST_PROMPT_3 = "rock, pop, dance i coś do tańca, zależy mi na maksymalnej energii, post rock, alternative rock, rock alternatywny, pop"
TEST_PROMPT_4 = "muzyka rockowa z lat 90, z klimatem podróży, postpankowa"
TEST_PROMPT_5 = "muzyka bez słów, smutna, nostalgiczna, spokojna"
TEST_PROMPT_6 = "szybka, intensywna, bardzo dobra do tańca, zabawy, zajsta do tańca, idealna do tańca, "
TEST_PROMPT_7 = "albo zwykły rock, albo jakiś post rock albo punk, coś takiego"
TEST_PROMPT_8 = "muzyka dynamiczna, szybko, szybkie tempo, wysokie tempo, energiczna"
TEST_PROMPT_9 = "muzyka, która koi nerwy"
TEST_PROMPT_10 = "muzyka, która koi nerwy"
TEST_PROMPT_11 = "Szukam muzyki rockowej, ale nie smutnej"
TEST_PROMPT_12 = "I want energetic songs, no slow music"
TEST_PROMPT_13 = "Chcę wesołe piosenki, coś szybkiego, ruchliwego, rock, muzyka popowa, z lat 90, dużo gitary, shoegaze"
TEST_PROMPT_14 = "szybki rock"

picked_prompt = TEST_PROMPT_8


try:
    lang_code = detect(picked_prompt)
except:
    lang_code = 'pl'

if lang_code == 'en':
    current_nlp = nlp_en
    current_matcher = matcher_en
    lang_txt = 'EN'
else:
    current_nlp = nlp_pl
    current_matcher = matcher_pl
    lang_txt = 'PL'


print(lang_txt)
relevant_phrases = extract_relevant_phrases(picked_prompt, current_nlp, current_matcher)

PL
Prompt: 'muzyka dynamiczna, szybko, szybkie tempo, wysokie tempo, energiczna' 
-> ['dynamiczna', 'szybkie tempo', 'energiczna', 'wysokie tempo']


## Dzielimy znalezione frazy na audio i tag

In [ ]:
LABELS_CONFIG = {
    # TAG
    "music_genre": {
        "desc": "music genre, gatunek muzyczny, category, style, \
        rock, pop, jazz, hip hop, rap, rapowa, metal, indie, alternative, \
        electronic, classical, reggae, techno, house, folk, blues, \
        funk, soul, punk, grunge, dubstep, dnb, disco, k-pop, \
        r&b, experimental, eksperymentalna, lo-fi, ambient",
        "route": "TAGS"
    },

    "song_type": {
        "desc": "soundtrack, ost, film score, remix, cover, \
        live performance, concert, instrumental, acoustic, unplugged, \
        opening, ending, wersja instrumentalna, na żywo",
        "route": "TAGS"
    },

    "musical_instrument": {
        "desc": "instrument, piano, pianino, guitar, gitara, drums, perkusja, \
        violin, skrzypce, bass, bas, saxophone, saksofon, synthesizer, syntezator",
        "route": "TAGS"
    },

    "vocal_characteristics": {
        "desc": "vocals, voice type, female vocalist, wokalistka, \
        male vocalist, wokalista, męski głos, kobiecy głos, choir, chór",
        "route": "TAGS"
    },

    # "geographical_location": {
    #     "desc": "country, origin, language, polish, polska, polski, \
    #     american, usa, british, uk, french, francuska, german, spanish, \
    #     russian, japanese, korean",
    #     "route": "TAGS"
    # },

    # "time_period": {
    #     "desc": "era, decade, year, 80s, 90s, 00s, 2020s, 70s, 60s, \
    #     lata 80, lata 90, lata 70, oldies, retro, klasyk",
    #     "route": "TAGS"
    # },

    # AUDIO
    "audio_emotion": {
        "desc": "emotion, feelings, uczucia, \
        happy, wesoła, wesoły, joyful, radosna, positive, pozytywna, \
        sad, smutna, smutny, depressive, depresyjna, crying, płaczliwa, \
        angry, agresywna, zła, fear, straszna, emotional, uczuciowa, \
        euphoric, euforyczna",
        "route": "AUDIO"
    },

    "audio_mood": {
        "desc": "mood, vibe, atmosphere, klimat, \
        chill, chillout, relaksująca, relaxing, calm, spokojna, peaceful, \
        dark, mroczna, gloomy, ponura, \
        romantic, romantyczna, love, miłosna, sensual, zmysłowa, \
        energetic, energetyczna, dance, taniec, party, imprezowa, epic, epicka, \
        dreamy, marzycielska, mysterious, tajemnicza",
        "route": "AUDIO"
    },

    "audio_technical": {
        "desc": "tempo, speed, volume, rhythm, \
        fast, szybka, szybkie tempo, high bpm, \
        slow, wolna, wolne tempo, downtempo, \
        dynamic, dynamiczna, rhythmic, rytmiczna, groove, \
        loud, głośna, noisy, hałaśliwa, \
        quiet, cicha, soft, delikatna, gentle, łagodna, \
        heavy, ciężka, mocna, hard, \
        acoustic, akustyczna, synthesized, syntetyczna",
        "route": "AUDIO"
    },

    "usage_context": {
        "desc": "activity, occasion, purpose, \
        focus, study, nauka, reading, coding, work, praca, \
        sleep, sen, relax, relaks, yoga, meditation, medytacja, \
        workout, gym, siłownia, running, bieganie, sport, training, \
        car, driving, auto, podróż, commute, spacer, \
        party, impreza, club, klub, dancing, taniec, \
        cleaning, sprzątanie, cooking, gotowanie, background, tło, \
        gaming, gry, playing, granie, stream, \
        date, randka, romance, miłość, sex, seks",
        "route": "AUDIO"
    },
}


GLINER_LABELS = [f"{k} ({v['desc']})" for k, v in LABELS_CONFIG.items()]
ROUTING_MAP = {k: v['route'] for k, v in LABELS_CONFIG.items()}

In [ ]:
GENRE_PHRASES_EXACT = {
    "rap": ["hip hop", "hip-hop", "hiphop", "new school", "old school"],
    "electronic": ["drum and bass", "dnb", "drum & bass", "psy trance"],
    "rock": ["post rock", "post-rock", "alt rock", "punk rock"],
    "pop": ["k-pop", "kpop", "synth pop"],
    "rnb": ["r&b", "rnb"]
}

GENRE_LEMMA_CONFIG = {
    "rap": ["rap", "rapowy", "rapsy", "trap", "drill"],
    "rock": ["rock", "rockowy", "metal", "metalowy", "grunge", "grungowy", "punk", "punkowy", "alt"],
    "pop": ["pop", "popowy", "disco", "mainstream", "radiowy", "przebój", "hit"],
    "electronic": ["elektroniczny", "elektro", "techno", "house", "trance", "dubstep", "edm", "klubowy"],
    "jazz": ["jazz", "jazzowy", "dżez", "blues", "bluesowy", "funk", "funkowy", "soul", "soulowy"],
    "classical": ["klasyczny", "orkiestrowy", "symfoniczny", "classical", "ochestra", "symphonic"],
    "reggae": ["reggae", "rasta", "ska", "dub"],
    "folk": ["folk", "folkowy", "etno", "ludowy", "country"],
    "indie": ["indie", "alternatywny"]
}

LEMMA_TO_GENRE_MAP = {}
for genre, keywords in GENRE_LEMMA_CONFIG.items():
    for word in keywords:
        LEMMA_TO_GENRE_MAP[word] = genre

In [ ]:
LANGUAGE_CONFIG = {
    "polish":   ["polski", "polska", "poland", "pl", "rodzimy", "krajowy", "polskie", "polsku"],
    "british": ["brytyjski", "brytania", "anglia", "londyn", "szkocki", "walijski","british", "uk", "england", "brit", "scotland", "wales", "london"],
    "american": ["amerykański", "ameryka", "usa", "stany", "zjednoczone","american", "america", "us", "states"],
    "english": ["angielski", "english", "anglojęzyczny", "angielsku"],
    "german":   ["niemiecki", "niemcy", "german", "deutsch", "germany", "niemiecku"],
    "french":   ["francuski", "francja", "french", "france", "francusku"],
    "spanish":  ["hiszpański", "hiszpania", "spanish", "latino", "latynoski", "spain", "hiszpańsku"],
    "italian":  ["włoski", "włochy", "italia", "italian", "italy", "włosku"],
    "russian":  ["rosyjski", "rosja", "ruski", "moskiewski", "russian", "russia", "rosyjsku"],
    "korean":   ["koreański", "korea", "korean", "koreańsku"],
    "japanese": ["japoński", "japonia", "japan", "anime", "japanese", "japońsku"],
    "swedish":  ["szwedzki", "szwecja", "swedish", "sweden", "szwedzku"],
    "foreign": ["zagraniczny", "obcy", "międzynarodowy", "światowy", "foreign", "international", "overseas","global"],
}


LEMMA_TO_TAG_MAP = {}
for tag, keywords in LANGUAGE_CONFIG.items():
    for word in keywords:
        LEMMA_TO_TAG_MAP[word] = tag

In [ ]:
def get_label_config_lists(config):
    """
    Przetwarza konfigurację etykiet, tworząc listę sformatowanych stringów dla modelu GLiNER
    oraz mapę powrotną do oryginalnych kluczy.

    Args:
        config (Dict[str, Dict[str, str]]): Słownik konfiguracyjny, w którym:
            - Kluczem (str) jest nazwa kategorii (np. 'gatunek_muzyczny').
            - Wartością (dict) jest słownik zawierający co najmniej klucz 'desc'
              z opisem słownym (np. 'rock, pop...').

    Returns:
        Tuple[List[str], Dict[str, str]]: Krotka zawierająca dwa elementy:
            1. gliner_labels (List[str]): Lista sformatowanych etykiet w postaci "Klucz (Opis)".
               Przykład: ["gatunek_muzyczny (rock, pop...)", ...].
            2. label_mapping (Dict[str, str]): Słownik mapujący pełną etykietę z powrotem na klucz.
               Przykład: {"gatunek_muzyczny (rock, pop...)": "gatunek_muzyczny"}.
    """
    gliner_labels = []
    label_mapping = {}

    for key, value in config.items():
        full_label = f"{key} ({value['desc']})"
        gliner_labels.append(full_label)
        label_mapping[full_label] = key

    return gliner_labels, label_mapping

GLINER_LABELS_LIST, GLINER_LABEL_MAP = get_label_config_lists(LABELS_CONFIG)

In [ ]:
def _match_geographical_location(doc_temp, fuzzy_cutoff):
    """
    Sprawdza dopasowanie do lokalizacji geograficznej (język/kraj).

    Returns:
        dict lub None: Słownik z category i route jeśli znaleziono, None w przeciwnym razie.
    """
    for token in doc_temp:
        lemma = token.lemma_.lower()

        found_key = None
        if lemma in LEMMA_TO_TAG_MAP:
            found_key = lemma
        elif len(lemma) > 4:
            matches = difflib.get_close_matches(lemma, LEMMA_TO_TAG_MAP.keys(), n=1, cutoff=fuzzy_cutoff)
            if matches:
                found_key = matches[0]
                print(f"Naprawiono literówkę: '{lemma}' -> '{found_key}'")

        if found_key:
            return {
                "category": "geographical_location",
                "route": "TAGS"
            }

    return None


def _match_music_genre_exact(phrase_lower):
    """
    Sprawdza dokładne dopasowanie do popularnych gatunków muzycznych.

    Returns:
        dict lub None: Słownik z category i route jeśli znaleziono, None w przeciwnym razie.
    """
    for genre_key, phrases in GENRE_PHRASES_EXACT.items():
        for exact_phrase in phrases:
            if exact_phrase in phrase_lower:
                return {
                    "category": "music_genre",
                    "route": "TAGS"
                }
    return None


def _match_music_genre_lemma(doc_temp, fuzzy_cutoff):
    """
    Sprawdza dopasowanie gatunku muzycznego przez lematyzację.

    Returns:
        dict lub None: Słownik z category i route jeśli znaleziono, None w przeciwnym razie.
    """
    for token in doc_temp:
        lemma = token.lemma_.lower()

        if lemma in LEMMA_TO_GENRE_MAP:
            return {
                "category": "music_genre",
                "route": "TAGS"
            }

        # fuzzy match
        elif len(lemma) > 4:
            matches = difflib.get_close_matches(lemma, LEMMA_TO_GENRE_MAP.keys(), n=1, cutoff=fuzzy_cutoff)

            if matches:
                found_key = matches[0]
                print(f"Naprawiono literówkę gatunku: '{lemma}' -> '{found_key}'")
                return {
                    "category": "music_genre",
                    "route": "TAGS"
                }

    return None


def _match_time_period(phrase_lower, doc_temp):
    """
    Sprawdza dopasowanie do okresu czasu/dekady.

    Returns:
        dict lub None: Słownik z category i route jeśli znaleziono, None w przeciwnym razie.
    """
    decade_keywords = ["lat", "rok", "80", "90", "00", "70", "60", "19", "20"]
    has_digit = any(char.isdigit() for char in phrase_lower)

    if has_digit and any(x in phrase_lower for x in decade_keywords):
        return {
            "category": "okres_czasu",
            "route": "TAGS"
        }

    era_lemmas = [
        "stary", "nowy", "old", "oldies", "klasyk", "classic",
        "retro", "vintage", "new", "newschool", "oldschool", "współczesny"
    ]

    for token in doc_temp:
        lemma = token.lemma_.lower()
        text = token.text.lower()

        if lemma in era_lemmas or text in era_lemmas:
            return {
                "category": "okres_czasu",
                "route": "TAGS"
            }

    return None


def _match_with_gliner(phrase_lower, gliner_predictions):
    """
    Dopasowuje frazę do encji wykrytych przez GLiNER.

    Returns:
        dict lub None: Słownik z category i route jeśli znaleziono, None w przeciwnym razie.
    """
    for entity in gliner_predictions:
        entity_lower = entity['text'].lower().strip()

        if phrase_lower in entity_lower or entity_lower in phrase_lower:
            full_label = entity['label']
            short_key = GLINER_LABEL_MAP.get(full_label)

            if short_key:
                return {
                    "category": short_key,
                    "route": LABELS_CONFIG[short_key]['route']
                }

    return None


def _classify_single_phrase(phrase, doc_temp, gliner_predictions, fuzzy_cutoff):
    """
    Klasyfikuje pojedynczą frazę przez serię dopasowań.

    Returns:
        dict: Słownik z phrase, category i route.
    """
    phrase_lower = phrase.lower().strip()

    # POPULARNE GATUNKI - dokładne dopasowanie
    match = _match_music_genre_exact(phrase_lower)
    if match:
        return {
            "phrase": phrase,
            "category": match["category"],
            "route": match["route"]
        }

    # POPULARNE GATUNKI - lematyzacja
    match = _match_music_genre_lemma(doc_temp, fuzzy_cutoff)
    if match:
        return {
            "phrase": phrase,
            "category": match["category"],
            "route": match["route"]
        }

    # CZAS/DEKADA
    match = _match_time_period(phrase_lower, doc_temp)
    if match:
        return {
            "phrase": phrase,
            "category": match["category"],
            "route": match["route"]
        }

    # JĘZYK/KRAJ
    match = _match_geographical_location(doc_temp, fuzzy_cutoff)
    if match:
        return {
            "phrase": phrase,
            "category": match["category"],
            "route": match["route"]
        }

    # GLINER
    match = _match_with_gliner(phrase_lower, gliner_predictions)
    if match:
        return {
            "phrase": phrase,
            "category": match["category"],
            "route": match["route"]
        }

    # FALLBACK
    return {
        "phrase": phrase,
        "category": "none",
        "route": "AUDIO"
    }


def classify_phrases_with_gliner(prompt, spacy_phrases, model, nlp_model, threshold=0.3, fuzzy_cutoff=0.9):
    """
    Klasyfikuje frazy wykryte przez spaCy, dopasowując je do encji wykrytych przez model GLiNER
    w kontekście całego promptu.

    Funkcja łączy precyzyjne wycinanie fraz (spaCy) z rozumieniem kontekstu (GLiNER).
    Opiera się na globalnych zmiennych konfiguracyjnych: GLINER_LABELS_LIST, GLINER_LABEL_MAP oraz LABELS_CONFIG.

    Args:
        prompt (str): Pełna treść zapytania użytkownika (niezbędna dla kontekstu GLiNERa).
        spacy_phrases (List[str]): Lista fraz (noun chunks/entities) wyekstrahowanych wcześniej przez spaCy.
        model (Any): Załadowany model GLiNER (np. obiekt klasy GLiNER).
        nlp_model: Załadowany model spaCy do lematyzacji.
        threshold (float, optional): Próg pewności dla predykcji GLiNERa. Domyślnie 0.3.
        fuzzy_cutoff (float, optional): Próg dla fuzzy matching. Domyślnie 0.8.

    Returns:
        List[Dict[str, str]]: Lista słowników, gdzie każdy słownik reprezentuje sklasyfikowaną frazę:
            {
                "phrase": str (oryginalna fraza ze spaCy),
                "category": str (klucz kategorii np. 'gatunek_muzyczny' lub 'cecha_audio'),
                "route": str (typ routingu np. 'TAGS' lub 'AUDIO')
            }
    """
    if not spacy_phrases or nlp_model is None:
        return []

    # Uruchamiamy GLiNER na całym tekście
    gliner_predictions = model.predict_entities(prompt, GLINER_LABELS_LIST, threshold=threshold)

    results = []

    for phrase in spacy_phrases:
        phrase_lower = phrase.lower().strip()
        doc_temp = nlp_model(phrase_lower)

        result = _classify_single_phrase(phrase, doc_temp, gliner_predictions, fuzzy_cutoff)
        results.append(result)

    return results

In [ ]:
classified_phrases = classify_phrases_with_gliner(
    "Muzyka do treningu szybka i energetyczna siłownia, rock mocna, po rosyjsku",
     ['rock', 'siłownia', 'do treningu', 'energetyczna', 'mocna', 'po rosyjsku', 'szybka'],
    model_gliner, current_nlp, threshold=0.2, fuzzy_cutoff=EXTRACTION_CONFIG["fuzzy_cutoff"])

classified_phrases

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'phrase': 'rock', 'category': 'music_genre', 'route': 'TAGS'},
 {'phrase': 'siłownia', 'category': 'musical_instrument', 'route': 'TAGS'},
 {'phrase': 'do treningu', 'category': 'none', 'route': 'AUDIO'},
 {'phrase': 'energetyczna', 'category': 'none', 'route': 'AUDIO'},
 {'phrase': 'mocna', 'category': 'none', 'route': 'AUDIO'},
 {'phrase': 'po rosyjsku',
  'category': 'geographical_location',
  'route': 'TAGS'},
 {'phrase': 'szybka', 'category': 'none', 'route': 'AUDIO'}]

### gliner tests

#### test1 - głównie gatunki, cechy audio

In [ ]:
test_cases = [
    {
        "text": "muzyka rockowa, grungowa, ale też może być jazz, najlepiej smutna, spokojna, eksperymentalna",
        "phrases": ["rockowa", "grungowa", "jazz", "smutna", "spokojna", "eksperymentalna"]
    },
    {
        "text": "Szukam czegoś w stylu alternatywna i elektroniczna, bardzo wesoła i taneczna",
        "phrases": ["alternatywna", "elektroniczna", "wesoła", "taneczna"]
    },
    {
        "text": "Gra na pianinie, momentami bardzo szybkie tempo, a potem wolne",
        "phrases": ["pianinie", "szybkie", "wolne"]
    },
    {
        "text": "Mroczna muzyka w tło, bardzo cicha i delikatna",
        "phrases": ["mroczna", "tło", "cicha", "delikatna"]
    },
    {
        "text": "Polska muzyka z lat 80, stary rock",
        "phrases": ["polska", "lat 80", "stary rock"]
    }
]

print("--- ROZPOCZYNAM TESTY ---")

for case in test_cases:
    print(f"\nPROMPT: {case['text']}")
    results = classify_phrases_with_gliner(
        case['text'],
        case['phrases'],
        model_gliner,
        current_nlp,
        threshold=0.2
    )

    for r in results:
        route_icon = "[TAG] " if r['route'] == "TAGS" else "[AUDIO]"
        print(f"  -> '{r['phrase']}': {route_icon} [{r['category']}]")

#### test 2 - języki, lokalizacje

In [ ]:
more_test_cases = [
    {
        "text": "Lubię muzykę francuską, może też być włoska opera",
        "phrases": ["francuską", "włoska"]
    },
    {
        "text": "Szukam piosenek tylko po polsku, ewentualnie po angielsku, żadnych instrumentalnych",
        "phrases": ["po polsku", "po angielsku", "instrumentalnych"]
    },
    {
        "text": "Japońska muzyka z anime albo coś z Korei, np k-pop",
        "phrases": ["japońska", "Korei", "k-pop"]
    },
    {
        "text": "Klasyczny rock z USA albo brytyjski punk, lata 90",
        "phrases": ["USA", "brytyjski", "punk", "lata 90"]
    },
    {
        "text": "Romantyczna hiszpańska gitara, spokojna i cicha",
        "phrases": ["hiszpańska", "gitara", "romantyczna", "spokojna", "cicha"]
    },
    {
        "text": "Interesuje mnie muzyka zagraniczna, najlepiej klimaty latino i do tańca",
        "phrases": ["zagraniczna", "latino", "do tańca"]
    },
    {
        "text": "Ciężka niemiecka elektronika albo smutna rosyjska ballada",
        "phrases": ["ciężka", "niemiecka", "elektronika", "smutna", "rosyjska"]
    }
]

# all_tests = test_cases + more_test_cases
all_tests = more_test_cases

print(f"--- ROZPOCZYNAM ROZSZERZONE TESTY (Liczba przypadków: {len(all_tests)}) ---")

for i, case in enumerate(all_tests):
    print(f"\n[{i+1}] PROMPT: {case['text']}")
    results = classify_phrases_with_gliner(
        case['text'],
        case['phrases'],
        model_gliner,
        current_nlp,
        threshold=0.15
    )

    for r in results:
        if r['route'] == "TAGS":
            icon = "[TAG]"
        else:
            icon = "[AUDIO]"

        print(f"   -> '{r['phrase']:<15}: {icon} [{r['category']}]")

#### test 3 - czynności

In [ ]:
test_cases_activities = [
    {
        "text": "Szukam muzyki na siłownię, ciężki trening, mocny bass, do podnoszenia ciężarów",
        "phrases": ["na siłownię", "ciężki trening", "mocny bass", "do podnoszenia ciężarów"]
    },
    {
        "text": "Energetyczny rock do biegania, szybkie tempo, jogging w parku",
        "phrases": ["energetyczny", "rock", "do biegania", "jogging"]
    },
    {
        "text": "Nocna jazda samochodem, amerykański rap, night drive, trasa",
        "phrases": ["nocna jazda", "samochodem", "amerykański", "rap", "night drive"]
    },
    {
        "text": "Spokojna muzyka do spania, głęboki sen, medytacja, ciche pianino",
        "phrases": ["do spania", "głęboki sen", "medytacja", "ciche", "pianino"]
    },
    {
        "text": "Muzyka do pracy, głębokie skupienie, coding, lo-fi beats, nauka",
        "phrases": ["do pracy", "skupienie", "coding", "lo-fi", "nauka"]
    },
    {
        "text": "Muzyka na domówkę, hity lata 80, disco, wesoła, do tańca",
        "phrases": ["na domówkę", "lata 80", "disco", "wesoła", "do tańca"]
    },
    {
        "text": "Polski hip hop na trening, agresywny i szybki",
        "phrases": ["polski", "hip hop", "na trening", "agresywny", "szybki"]
    },
    {
        "text": "Muzyka grungowa do samochodu",
        "phrases": ["grungowa", "do samochodu"]
    }
]

print("\n--- ROZPOCZYNAM TESTY (AKTYWNOŚCI & KONTEKST) ---")

for case in test_cases_activities:
    print(f"\nPROMPT: {case['text']}")
    results = classify_phrases_with_gliner(
        case['text'],
        case['phrases'],
        model_gliner,
        current_nlp,
        threshold=0.15,
        fuzzy_cutoff=0.8
    )

    for r in results:
        # Wizualizacja wyniku
        if r['route'] == "TAGS":
            route_icon = "✅ [TAG]"
            color = "\033[92m"
        else:
            route_icon = "🎧 [AUDIO]"
            color = "\033[94m"

        end_color = "\033[0m"

        meta_info = f" {r.get('meta', {})}" if r.get('meta') else ""

        print(f"  -> '{r['phrase']}': {color}{route_icon} [{r['category']}]{end_color}{meta_info}")

#### test 4 - dziwne gatunki muzyczne

In [ ]:
test_cases_niche = [
    {
        "text": "Szukam ciężkiego djentu na siłownię, agresywne riffy",
        "phrases": ["ciężkiego", "djentu", "na siłownię", "agresywne riffy"]
    },
    {
        "text": "Shoegaze do nauki i czytania, ściana dźwięku ale spokojna",
        "phrases": ["shoegaze", "do nauki", "czytania", "ściana dźwięku", "spokojna"]
    },
    {
        "text": "Nocna jazda autem, vaporwave i synthwave, neonowe światła",
        "phrases": ["nocna jazda", "autem", "vaporwave", "synthwave", "neonowe światła"]
    },
    {
        "text": "Skomplikowany math rock, zmienne metrum, techniczne granie",
        "phrases": ["skomplikowany", "math rock", "zmienne metrum", "techniczne granie"]
    },
    {
        "text": "Darkwave do biegania w nocy, szybkie tempo, mroczny klimat",
        "phrases": ["darkwave", "do biegania", "w nocy", "szybkie tempo", "mroczny klimat"]
    },
    {
        "text": "Relaks przy afrobeat, chillout, pozytywne wibracje",
        "phrases": ["relaks", "afrobeat", "chillout", "pozytywne wibracje"]
    },
    {
        "text": "Japoński city pop z lat 80",
        "phrases": ["japoński", "city pop", "z lat 80"]
    }
]

print("\n--- ROZPOCZYNAM TESTY (NISZOWE GATUNKI + BLOCKLISTA) ---")

for case in test_cases_niche:
    print(f"\nPROMPT: {case['text']}")
    results = classify_phrases_with_gliner(
        case['text'],
        case['phrases'],
        model_gliner,
        current_nlp,
        threshold=0.15,
        fuzzy_cutoff=0.8
    )

    for r in results:
        # Wizualizacja wyniku
        if r['route'] == "TAGS":
            route_icon = "✅ [TAG]"
            color = "\033[92m"
        else:
            route_icon = "🎧 [AUDIO]"
            color = "\033[94m"

        end_color = "\033[0m"

        meta_info = f" {r.get('meta', {})}" if r.get('meta') else ""

        print(f"  -> '{r['phrase']}': {color}{route_icon} [{r['category']}]{end_color}{meta_info}")

### e5 preparation

In [ ]:
def prepare_queries_for_e5_separated(classified_data, original_prompt):
    """
    Rozdziela frazy na Tagi i Audio.
    Zwraca słownik z dwoma listami: "TAGS" i "AUDIO".
    """

    return {
        "TAGS": [
            item['phrase'] for item in classified_data
            if item['route'] == 'TAGS'
        ],
        "AUDIO": [
            item['phrase'] for item in classified_data
            if item['route'] == 'AUDIO'
        ]
    }

In [ ]:
e5_queries_separated = prepare_queries_for_e5_separated(classified_phrases, picked_prompt)
e5_queries_separated

## Dopasowanie audio

In [ ]:
FEATURE_DESCRIPTIONS = {
    'valence': [
        # ((Min, Max), "Opis")
        ((0.0, 0.25), "very low valence, very sad, melancholic, dark, gloomy emotional mood music - bardzo smutna, dołująca, ponura, depresyjna, mroczna"),
        ((0.25, 0.40), "low valence, bittersweet, thoughtful, introspective, moody emotional mood music - smutnawa, nostalgiczna, refleksyjna, nastrojowa, melancholijna"),
        # ((0.45, 0.60), "medium valence, neutral emotional mood, neither clearly happy nor clearly sad music"),
        ((0.70, 0.90), "high valence, positive, pleasant, warm, cheerful, uplifting emotional mood music - wesoła, pozytywna, radosna, przyjemna, ciepła, optymistyczna"),
        ((0.90, 1.0), "very high valence, very happy, joyful, exstatic, euphoric, bright, feel-good emotional mood music - bardzo wesoła, euforyczna, ekstatyczna, pełna radości, szczęśliwa")
    ],

    'danceability': [
        ((0.0, 0.25), "very low danceability, not danceable, abstract or experimental, weak or irregular rhythm music - nie do tańca, nietaneczna, nieregularny rytm, abstrakcyjna, bez rytmu"),
        ((0.25, 0.4), "low danceability, little groove, minimal rhythm, not primarily for dancing music - mało taneczna, słaby rytm, raczej do słuchania niż tańczenia"),
        # ((0.50, 0.70), "medium danceability, some groove, steady rhythm music"),
        ((0.80, 0.90), "high danceability, clear beat, strong groove, good for dancing, club-oriented music - taneczna, do tańca, klubowa, dobry rytm, bujająca"),
        ((0.90, 1.0), "very high danceability, strong groove, infectious rhythm, perfect for dancing, party, club banger music - bardzo taneczna, imprezowa, parkietowa, porywająca do tańca, wixa")
    ],

    'acousticness': [
        ((0.0, 0.05), "very low acousticness, fully electronic, synthetic, digital, computer-generated sound music - w pełni elektroniczna, syntetyczna, cyfrowa, syntezatory, techno brzmienie"),
        ((0.05, 0.25), "low acousticness, mostly electronic with some subtle organic or acoustic elements music - głównie elektroniczna, nowoczesne brzmienie"),
        # ((0.45, 0.65), "medium acousticness, balanced mix of acoustic and electronic instruments, hybrid sound music"),
        ((0.65, 0.85), "high acousticness, mostly acoustic, organic, live instruments such as accoustic guitar or piano music - akustyczna, naturalna, żywe instrumenty, gitara, pianino"),
        ((0.85, 1.0), "very high acousticness, fully acoustic, unplugged, natural, organic instruments only music - w pełni akustyczna, bez prądu, unplugged, naturalne brzmienie")
    ],

    'n_tempo': [
        ((0.0, 0.30), "very slow tempo, very slow pace, dragging rhythm music - bardzo wolne tempo, bardzo wolna, ślimacze tempo, ciągnąca się"),
        ((0.30, 0.45), "slow tempo, downtempo, slow pace, relaxed rhythm music - wolne tempo, wolna, spokojny rytm, powolna"),
        ((0.45, 0.7), "medium tempo, moderate pace, walking pace music - średnie tempo, umiarkowana szybkość, normalne tempo"),
        ((0.70, 0.90), "fast tempo, uptempo, quick pace, energetic rhythm music - szybkie tempo, szybka, żwawa, energiczny rytm"),
        ((0.90, 1.0), "very fast tempo, rapid pace, racing rhythm, frantic speed music - bardzo szybkie tempo, bardzo szybka, pędząca, zawrotna prędkość")
    ],

    'instrumentalness': [
        ((0.0, 0.35), "very low instrumentalness, strong presence of vocals and lyrics, clear singing, vocal-focused track - wokalna, z wokalem, śpiewana, piosenka z tekstem, głos"),
        ((0.35, 0.75), "medium instrumentalness, mix of vocals and instrumental sections, vocals present but not constant - mieszana, trochę śpiewania trochę muzyki"),
        ((0.75, 1.0), "very high instrumentalness, fully instrumental track, no vocals, no singing, music without lyrics - instrumentalna, bez słów, bez wokalu, sama muzyka, melodia")
    ],

    'energy': [
        ((0.0, 0.25), "very low energy, motionless, static, sleep-inducing, minimal activity music - bardzo niska energia, statyczna, usypiająca, bez energii, leniwa"),
        ((0.25, 0.45), "low energy, relaxed, laid-back, mellow, slow-moving atmosphere music - niska energia, zrelaksowana, luźna, spokojna, chillout"),
        ((0.45, 0.70), "medium energy, moderate pace, steady rhythm, balanced activity music - średnia energia, umiarkowana, zrównoważona"),
        ((0.70, 0.90), "high energy, active, driving rhythm, fast-paced, stimulating, busy arrangement music - wysoka energia, energetyczna, żywa, pobudzająca, mocna"),
        ((0.90, 1.0), "very high energy, hyperactive, restless, frantic, adrenaline-pumping, non-stop action music - bardzo wysoka energia, wybuchowa, szalona, adrenalina, ogień, pompa")
    ],

    'n_loudness': [
        ((0.0, 0.25), "very low loudness, barely audible, near silence, whisper-like volume, extremely quiet music - bardzo cicha, ledwo słyszalna, szept, cisza"),
        ((0.25, 0.50), "low loudness, soft volume, background level, reduced amplitude, delicate sound music - cicha, delikatna, w tle, miękkie brzmienie"),
        ((0.50, 0.75), "medium loudness, standard volume, normal mastering level music - normalna głośność, standardowa"),
        ((0.75, 0.90), "high loudness, loud volume, amplified sound, noisy, high amplitude music - głośna, hałaśliwa, mocne brzmienie"),
        ((0.90, 1.0), "very high loudness, maximum volume, deafening, high decibels music - bardzo głośna, ogłuszająca, maksymalna głośność, huk")
    ],

    'speechiness': [
        # Speechiness > 0.66 to zazwyczaj podcasty, 0.33-0.66 to rap, < 0.33 to muzyka
        ((0.0, 0.22), "very low speechiness, purely musical track, no spoken words, fully melodic music - muzyka, melodia, śpiew, mało gadania"),
        ((0.22, 0.66), "low speechiness, mostly music with occasional spoken words or short background phrases - muzyka ze wstawkami mowy, rap, hip-hop"),
        ((0.66, 1.0), "medium to high speechiness, balanced mix of speech and music, frequent spoken segments, rap-like or talky structure - dużo gadania, mowa, wywiad, audiobook, podcast, recytacja"),
    ],
}

In [ ]:
ACTIVITY_GROUPS = {
    'deep_focus': {
        'triggers': [
            # EN
            "reading", "reading books", "studying", "learning", "homework",
            "focus", "concentration", "deep work", "coding", "programming",
            "writing", "library", "chess", "brainstorming", "thinking",
            "exam preparation", "working", "office work", "study session",
            # PL
            "czytanie", "książki", "nauka", "uczenie się", "praca domowa",
            "skupienie", "koncentracja", "głęboka praca", "programowanie", "kodowanie",
            "pisanie", "biblioteka", "szachy", "myślenie", "egzamin", "sesja",
            "praca biurowa", "do nauki", "do pracy"
        ],
        'rules': [
            ('instrumentalness', (0.8, 1.0)),
            ('speechiness', (0.0, 0.2)),
            ('energy', (0.1, 0.5)),
            ('n_loudness', (0.0, 0.6))
        ]
    },

    'sleep_relax': {
        'triggers': [
            # EN
            "sleeping", "falling asleep", "insomnia", "nap", "napping",
            "meditation", "meditating", "yoga", "mindfulness", "zen",
            "spa", "massage", "calm down", "anxiety relief", "stress relief",
            "lying in bed", "winding down", "evening relaxation", "chill out",
            # PL
            "spanie", "sen", "zasypianie", "bezsenność", "drzemka",
            "medytacja", "joga", "uważność", "spa", "masaź",
            "spokój", "stres", "leżenie w łóżku", "wieczorny relaks",
            "odpoczynek", "wyciszenie", "do spania", "kołysanka"
        ],
        'rules': [
            ('energy', (0.0, 0.25)),
            ('n_tempo', (0.0, 0.35)),
            ('n_loudness', (0.0, 0.35)),
            ('acousticness', (0.5, 1.0)),
            ('valence', (0.4, 0.7))
        ]
    },

    'workout_intense': {
        'triggers': [
            # EN
            "gym", "weightlifting", "crossfit", "boxing", "kickboxing",
            "hiit", "interval training", "sprint", "running fast", "cardio",
            "beast mode", "motivation", "pump up", "hardcore training",
            "powerlifting", "bodybuilding", "marathon training",
            # PL
            "siłownia", "ciężary", "boks", "interwały", "sprint",
            "bieganie", "szybki bieg", "kardio", "motywacja", "trening",
            "mocny trening", "kulturystyka", "maraton", "ćwiczenia",
            "na siłkę", "pompa"
        ],
        'rules': [
            ('energy', (0.8, 1.0)),
            ('n_tempo', (0.7, 1.0)),
            ('n_loudness', (0.7, 1.0)),
            ('danceability', (0.5, 0.9))
        ]
    },

    'commute_jogging': {
        'triggers': [
            # EN
            "jogging", "walking", "walking the dog", "commuting", "driving",
            "road trip", "car ride", "night drive", "highway", "bus ride",
            "train ride", "traveling", "subway", "city walk", "bike riding",
            "cycling",
            # PL
            "jogging", "trucht", "spacer", "spacer z psem", "dojazd", "jazda autem",
            "samochód", "podróż", "nocna jazda", "autostrada", "autobus",
            "pociąg", "metro", "miasto", "rower", "jazda na rowerze",
            "kierowanie", "za kółkiem"
        ],
        'rules': [
            ('energy', (0.5, 0.8)),
            ('n_tempo', (0.45, 0.7)),
            ('valence', (0.4, 0.9))
        ]
    },

    'party_club': {
        'triggers': [
            # EN
            "party", "house party", "clubbing", "dancing", "dancefloor",
            "friday night", "saturday night", "birthday", "celebration",
            "drinking", "pre-game", "getting ready", "festival", "rave",
            "disco", "summer party", "pool party",
            # PL
            "impreza", "domówka", "klub", "taniec", "parkiet",
            "piątek wieczór", "sobota", "urodziny", "świętowanie",
            "picie", "bifor", "festiwal", "dyskoteka", "letnia impreza",
            "basen", "do tańca", "wixa"
        ],
        'rules': [
            ('danceability', (0.8, 1.0)),
            ('energy', (0.8, 1.0)),
            ('valence', (0.6, 1.0)),
            ('n_loudness', (0.7, 1.0))
        ]
    },

    'chores_background': {
        'triggers': [
            # EN
            "cleaning", "cleaning the house", "cooking", "kitchen",
            "doing dishes", "gardening", "chores", "housework",
            "morning coffee", "breakfast", "sunday morning",
            "hanging out", "friends coming over", "dinner party", "barbecue",
            # PL
            "sprzątanie", "porządki", "gotowanie", "kuchnia",
            "zmywanie", "ogród", "prace domowe", "obowiązki",
            "poranna kawa", "śniadanie", "niedziela rano",
            "spotkanie ze znajomymi", "obiad", "grill", "tło",
            "w tle", "do kawy"
        ],
        'rules': [
            ('energy', (0.4, 0.7)),
            ('valence', (0.5, 0.9)),
            ('danceability', (0.4, 0.8)),
            ('acousticness', (0.2, 0.8))
        ]
    },

    'sad_emotional': {
        'triggers': [
            # EN
            "sad", "crying", "depression", "depressed", "lonely",
            "heartbreak", "breakup", "missing someone", "rainy day",
            "melancholy", "grieving", "emotional", "moody", "nostalgia",
            "bad day",
            # PL
            "smutek", "płacz", "depresja", "samotność",
            "złamane serce", "rozstanie", "tęsknota", "deszczowy dzień",
            "melancholia", "żałoba", "emocje", "nostalgia",
            "zły dzień", "doła", "smutna"
        ],
        'rules': [
            ('valence', (0.0, 0.3)),
            ('energy', (0.0, 0.4)),
            ('danceability', (0.0, 0.4)),
            ('n_tempo', (0.0, 0.4))
        ]
    },

    'romance': {
        'triggers': [
            # EN
            "date night", "romantic dinner",
            "candlelight", "intimacy", "cuddling", "boyfriend", "girlfriend",
            "valentine", "sexy", "seduction", "late night", "bedroom",
            # PL
            "randka", "romantyczna kolacja",
            "świece", "nastrojowa", "intymność", "przytulanie", "chłopak", "dziewczyna",
            "walentynki", "seks", "sypialnia", "wieczór we dwoje", "miłość"
        ],
        'rules': [
            ('n_tempo', (0.1, 0.5)),
            ('danceability', (0.5, 0.8)),
            ('energy', (0.2, 0.6)),
            ('n_loudness', (0.2, 0.6))
        ]
    },

    'gaming': {
        'triggers': [
            # EN
            "gaming", "playing games", "esports", "streaming", "twitch",
            "league of legends", "fortnite", "fps", "rpg", "cyberpunk",
            "hacker", "futuristic",
            # PL
            "granie", "gry", "esport", "stream",
            "strzelanki", "haker", "futurystyczna", "do grania", "gierki"
        ],
        'rules': [
            ('energy', (0.7, 1.0)),
            ('acousticness', (0.0, 0.2)),
            ('speechiness', (0.0, 0.3))
        ]
    }
}

In [ ]:
def prepare_search_indices(model, feature_descriptions, activity_groups):
    """
    Inicjalizuje i generuje wektorowe indeksy wyszukiwania dla cech audio oraz aktywności.

    Proces polega na transformacji opisów tekstowych (natural language descriptions)
    na ustandaryzowane osadzenia wektorowe (embeddings) przy użyciu modelu E5.

    Args:
        model: Model SentenceTransformer (np. multilingual-e5).
        feature_descriptions (dict): Słownik definicji cech technicznych (np. tempo, energy).
        activity_groups (dict): Słownik grup aktywności wraz z ich triggerami.

    Returns:
        dict: Słownik zawierający wygenerowane macierze wektorowe podzielone na AUDIO i ACTIVITY.
    """
    print("[INIT] Generowanie indeksów wektorowych...")
    indices = {
        'AUDIO': {},
        'ACTIVITY': {}
    }

    # generowanie ideksow audio
    for key, descs in feature_descriptions.items():
        passages = [f"passage: {d[1]}" for d in descs]
        indices['AUDIO'][key] = model.encode(passages, normalize_embeddings=True)

    # Generowanie indeksów ACTIVITY
    for key, data in activity_groups.items():
        triggers_str = ", ".join(data['triggers'])
        desc_text = f"music for {triggers_str}"

        indices['ACTIVITY'][key] = model.encode([f"passage: {desc_text}"], normalize_embeddings=True)

    print(f"[SUCCESS] Zainicjalizowano {len(indices['AUDIO'])} cech audio i {len(indices['ACTIVITY'])} grup aktywności.")
    return indices

SEARCH_INDICES = prepare_search_indices(model_e5, FEATURE_DESCRIPTIONS, ACTIVITY_GROUPS)

In [ ]:
def phrases_to_features(phrases_list, search_indices, lang_code='pl'):
    """
    Mapuje listę fraz tekstowych na parametry techniczne (audio features) oraz aktywności,
    wykorzystując hybrydowe podejście oparte na analizie gramatycznej (NLP)
    oraz osadzeniach wektorowych (embeddings) modelu E5.

    Logika działania:
    1. Analiza gramatyczna (Routing): Każda fraza jest analizowana przez model spaCy.
       Na podstawie części mowy (POS) przypisywany jest zakres wyszukiwania:
       - ACTIVITY_ONLY: Dla przyimków i czasowników (np. "do spania", "biegać").
       - AUDIO_ONLY: Dla czystych przymiotników i przysłówków (np. "szybka").
       - BOTH: Dla rzeczowników i fraz mieszanych (np. "tempo", "sen").
    2. Wyszukiwanie semantyczne: Obliczane jest podobieństwo kosinusowe między wektorem
       zapytania a wektorami w wybranych indeksach (AUDIO/ACTIVITY).
    3. Turniej (Max Pooling): W trybie BOTH wygrywa kategoria o najwyższym współczynniku ufności.
    4. Konsolidacja (Merge): Parametry audio nadpisują domyślne ustawienia aktywności,
       jeśli ich pewność (confidence) jest wyższa.

    Args:
        phrases_list (list[str]): Lista fraz wyodrębnionych z promptu (np. ["szybka", "do nauki"]).
        search_indices (dict): Słownik zawierający wstępnie obliczone embeddingi dla
            kluczy 'AUDIO' i 'ACTIVITY'.
        lang_code (str, optional): Kod języka ('pl' lub 'en') decydujący o modelu spaCy
            i sufiksie kontekstowym. Domyślnie 'pl'.

    Returns:
        list[tuple]: Posortowana malejąco lista krotek (nazwa_cechy, dane), gdzie dane
            zawierają docelowy przedział wartości i współczynnik ufności.
            Przykład: [('n_tempo', {'value': (0.8, 0.9), 'confidence': 0.88})]
    """

    if not phrases_list: return []

    AUDIO_INDEX = search_indices['AUDIO']
    ACTIVITY_INDEX = search_indices['ACTIVITY']

    try:
        nlp_model = nlp_pl if lang_code == 'pl' else spacy.load("en_core_web_sm")
    except:
        nlp_model = nlp_pl

    found_explicit_audio = []
    found_activities = []

    print(f"\nFRAZY: {phrases_list}")

    for phrase in phrases_list:
        suffix = " muzyka" if lang_code == 'pl' else " music"
        context_vec = model_e5.encode([f"query: {suffix} {phrase}"], normalize_embeddings=True)

        # ANALIZA GRAMATYCZNA (NLP)
        doc = nlp_model(phrase.lower())
        first_token = doc[0]

        # Flagi gramatyczne
        is_activity_indicator = first_token.pos_ in ["ADP", "PART", "VERB"]
        is_adj_or_adv = any(t.pos_ in ["ADJ", "ADV"] for t in doc)
        has_noun = any(t.pos_ in ["NOUN", "PROPN"] for t in doc)

        # ROUTING
        if is_activity_indicator:
            search_scope = "ACTIVITY_ONLY"
        elif is_adj_or_adv and not has_noun:
            search_scope = "AUDIO_ONLY"
        else:
            search_scope = "BOTH"

        # WYSZUKIWANIE
        best_audio_key, best_audio_val, best_audio_score = None, None, 0.0
        best_act_key, best_act_score = None, 0.0

        # Szukamy w Audio
        if search_scope in ["AUDIO_ONLY", "BOTH"]:
            for feat, embs in AUDIO_INDEX.items():
                sims = cosine_similarity(context_vec, embs)[0]
                idx = np.argmax(sims)
                if sims[idx] > best_audio_score:
                    best_audio_score = sims[idx]
                    best_audio_key = feat
                    best_audio_val = FEATURE_DESCRIPTIONS[feat][idx][0]

        # Szukamy w Activity
        if search_scope in ["ACTIVITY_ONLY", "BOTH"]:
            for group, embs in ACTIVITY_INDEX.items():
                sims = cosine_similarity(context_vec, embs)[0]
                mx = np.max(sims)
                if mx > best_act_score:
                    best_act_score = mx
                    best_act_key = group

        # 4. WERDYKT (Max Pooling / Turniej)
        winner_type = None
        final_score = 0.0

        if search_scope == "AUDIO_ONLY":
            winner_type = 'AUDIO'; final_score = best_audio_score
        elif search_scope == "ACTIVITY_ONLY":
            winner_type = 'ACTIVITY'; final_score = best_act_score
        else:
            # TURNIEJ DLA "BOTH": Wygrywa wyższy confidence
            if best_audio_score > best_act_score:
                winner_type = 'AUDIO'; final_score = best_audio_score
            else:
                winner_type = 'ACTIVITY'; final_score = best_act_score

        # Dodawanie do tymczasowych list wyników
        if winner_type == 'AUDIO':
            print(f"[MATCH:AUDIO] '{phrase}' -> feature: {best_audio_key} (score: {final_score:.4f})")
            found_explicit_audio.append((best_audio_key, best_audio_val, final_score))
        else:
            print(f"[MATCH:ACTIVITY] '{phrase}' -> category: {best_act_key} (score: {final_score:.4f})")
            found_activities.append((best_act_key, final_score))

    # merge - audio ma priorytet przy konfliktach
    merged = {}

    # KROK A: Presety aktywności
    found_activities.sort(key=lambda x: x[1]) # od najsłabszej do najsilniejszej
    for group, score in found_activities:
        if group in ACTIVITY_GROUPS:
            for r_feat, r_val in ACTIVITY_GROUPS[group]['rules']:
                merged[r_feat] = {'value': r_val, 'confidence': float(score)}

    # KROK B: Cechy audio (nadpisują bazę, jeśli są silniejsze)
    for feat, val, score in found_explicit_audio:
        if feat not in merged or score > merged[feat]['confidence']:
            if feat in merged:
                  print(f"nadpisywanie '{feat}': (Score: {score:.2f})")
            merged[feat] = {'value': val, 'confidence': float(score)}

    return sorted([(k, v) for k, v in merged.items()], key=lambda x: x[1]['confidence'], reverse=True)

In [ ]:
audio_features_queries = e5_queries_separated['AUDIO']

try:
    lang = detect(picked_prompt)
except:
    lang = 'pl'

criteria_audio = phrases_to_features(audio_features_queries, SEARCH_INDICES, lang_code=lang)
criteria_audio

## Dopasowanie tagów

In [ ]:
df_tag_embeddings = pd.read_parquet("../df_unique_tag_embeddings.parquet")
df_tag_embeddings

Embeddingi tagów

In [ ]:
TAG_VECS = np.array(df_tag_embeddings["tag_embedding"].to_list(), dtype=np.float32)
TAGS = df_tag_embeddings["tag"].tolist()

In [ ]:
def map_phrases_to_tags(
    phrases: list[str],
    model,
    tag_vecs,
    tags_list,
    threshold_strict: float = 0.82,
    threshold_lenient: float = 0.75
) -> dict[str, float]:

    if not phrases: return {}

    # Obliczamy wektory E5
    q_vecs = model.encode(
        [f"query: {p}" for p in phrases],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")

    sims_matrix = cosine_similarity(tag_vecs, q_vecs)
    found_tags = {}

    for i, phrase in enumerate(phrases):
        phrase_scores = sims_matrix[:, i]
        best_idx = np.argmax(phrase_scores)
        best_score = float(phrase_scores[best_idx])
        best_tag_name = tags_list[best_idx]

        is_match = False
        reason = ""

        # wynik wysoki - e5
        if best_score >= threshold_strict:
            is_match = True
            reason = "High Score"

        # wynik jest średni, ale tekst się pokrywa (tag zawiera się we frazie lub odwrotnie)
        elif best_score >= threshold_lenient:
            p_lower = phrase.lower()
            t_lower = best_tag_name.lower()

            if t_lower in p_lower or p_lower in t_lower:
                is_match = True
                reason = "Text Match Rescue"

            # fuzzy match
            elif difflib.SequenceMatcher(None, p_lower, t_lower).ratio() > 0.7:
                 is_match = True
                 reason = "Fuzzy Rescue"

        if is_match:
            print(f"'{phrase}' -> '{best_tag_name}' ({best_score:.3f}) [{reason}]")
            if best_tag_name in found_tags:
                found_tags[best_tag_name] = max(found_tags[best_tag_name], best_score)
            else:
                found_tags[best_tag_name] = best_score
        else:
            print(f"'{phrase}' -> odrzucono '{best_tag_name}' ({best_score:.3f}) [Za niski score i brak podobieństwa tekstu]")

    return found_tags

In [ ]:
tags_queries = e5_queries_separated['TAGS']
print(tags_queries)
criteria_tags = map_phrases_to_tags(tags_queries, model=model_e5, tag_vecs=TAG_VECS, tags_list=TAGS, threshold_strict=EXTRACTION_CONFIG["tag_similarity_threshold"], threshold_lenient=EXTRACTION_CONFIG["tag_similarity_threshold_lenient"])
criteria_tags

### Testy tagów

#### test1

In [ ]:
test_cases = {
    "1. POLSKIE ODMIANY": [
        "rockowa", "metalowe", "jazzowy", "punkowa",
        "elektroniczna", "klasyczna", "popowe", "alternatywna"
    ],

    "2. GATUNKI KONKRETNE I NISZOWE": [
        "shoegaze", "vaporwave", "djent", "math rock",
        "synthwave", "k-pop", "reggae", "techno", "dubstep",
        "phonk", "lo-fi", "black metal"
    ],

    "3. INSTRUMENTY I NASTRÓJ": [
        "pianino", "na gitarze", "saksofon", "skrzypce",
        "smutna", "wesoła", "mroczna", "szybka", "spokojna"
    ],

    "4. PUŁAPKI": [
        "hydraulik", "przepis na ciasto", "odgłosy lasu", "siłownia"
    ]
}

for category, phrases_list in test_cases.items():
    print(f"\n🟦 {category}")

    map_phrases_to_tags(
        phrases=phrases_list,
        model=model_e5,
        tag_vecs=TAG_VECS,
        tags_list=TAGS,
        threshold_strict=0.0,
        threshold_lenient=0.0
    )

## Algorytm

uzyskujemy wagi wsyztskich tagów zsumowane do 1

In [ ]:
def get_query_tag_weights(
    raw_tags
) -> dict[str, float]:
    """
    Normalizuje wagi tak, by sumowały się do 1.
    """
    s = sum(raw_tags.values())
    if s <= 0:
        return raw_tags

    return {t: v / s for t, v in raw_tags.items()}

In [ ]:
query_tag_weights = get_query_tag_weights(criteria_tags)
query_tag_weights

{'rock': 0.5189387808240806, 'russian': 0.48106121917591943}

In [ ]:
def score_songs_by_tags(
    query_tag_weights: dict[str, float],
    inv_index: dict[str, np.ndarray],
    n_songs: int,
    query_pow: float = 1.0,
) -> np.ndarray:
    """
    Liczy dopasowanie (score) wszystkich utworów w bazie do tagów z zapytania.
    Działa w oparciu o szybki odwrócony indeks (Inverted Index).

    Algorytm sumuje wagi tagów znalezionych w piosence, opcjonalnie mnożąc je przez IDF
    (rzadkość tagu), a na końcu normalizuje wynik do zakresu 0.0 - 1.0.

    Args:
        query_tag_weights (dict[str, float]): Znormalizowane wagi tagów wyciągnięte z promptu.
            Klucz: nazwa tagu (np. 'rock'), Wartość: waga (np. 0.6).
        inv_index (dict[str, np.ndarray]): Odwrócony indeks bazy utworów.
            Klucz: nazwa tagu. Wartość: tablica indeksów (ID) piosenek, które mają ten tag.
            Umożliwia błyskawiczne wyszukiwanie bez iteracji po wszystkich piosenkach.
        song_tag_count (np.ndarray): Wektor o długości równej liczbie piosenek w bazie (`n_songs`).
            Służy tutaj do określenia wymiaru wektora wyników.
        query_pow (float, optional): Wykładnik potęgi do wyostrzania wag z promptu.
            - 1.0: brak zmian (liniowo).
            - > 1.0 (np. 2.0): Zwiększa przepaść między ważnymi a mało ważnymi tagami z zapytania.
            Domyślnie 1.0.

    Returns:
        np.ndarray: Wektor o kształcie `(n_songs,)` z wartościami float w zakresie [0.0, 1.0].
            Indeks w wektorze odpowiada indeksowi piosenki w `df_songs`.
    """

    qtw = {}

    max_theoretical_score = 0.0

    for t, w in query_tag_weights.items():
        w_processed = w ** query_pow

        this_idf = 1.0
        if use_idf and idf_map is not None:
            this_idf = float(idf_map.get(t, 1.0))

        final_weight = w_processed * this_idf
        qtw[t] = final_weight
        max_theoretical_score += final_weight

    scores = np.zeros(n_songs, dtype=np.float32)

    # sumowanie punktow
    if max_theoretical_score > 0:
        for tag, weight in qtw.items():
            if tag not in inv_index:
                continue

            idxs = inv_index[tag]
            scores[idxs] += weight

        # normalizacja
        scores /= max_theoretical_score

    return np.clip(scores, 0.0, 1.0)

In [ ]:
scores_songs_tag = score_songs_by_tags(
    query_tag_weights=query_tag_weights,
    inv_index=INV_INDEX,
    n_songs=len(df_songs),
    use_idf=SCORING_CONFIG['use_idf'],
    query_pow=SCORING_CONFIG['query_pow']
)
scores_songs_tag

array([0.5189388, 0.5189388, 0.5189388, ..., 0.       , 0.       ,
       0.5189388], dtype=float32)

In [ ]:
def retrieve_candidates_tags(
    scores_songs_tag: np.ndarray,
    n_candidates: int = 400,
    flat_delta: float = 0.05
):
    """
    Wybiera kandydatów z bazy utworów na podstawie obliczonych punktów (score) za tagi.
    Stosuje mechanizm "miękkiego odcięcia" (soft cut-off) – jeśli utwory tuż za progiem n_candidates
    mają bardzo zbliżony wynik, również są dołączane do listy.

    Funkcja polega na globalnej zmiennej:
        - df_songs (pd.DataFrame): Główna baza danych utworów.

    Args:
        scores_songs_tag (np.ndarray): Wektor wyników dopasowania tagów dla wszystkich utworów
            (musi mieć tę samą długość co df_songs).
        n_candidates (int, optional): Docelowa liczba kandydatów do zwrócenia. Domyślnie 400.
        flat_delta (float, optional): Margines tolerancji dla wyników. Utwory poza top N,
            których wynik różni się od ostatniego zaakceptowanego o mniej niż ta wartość,
            zostaną również dołączone. Domyślnie 0.05.

    Returns:
        pd.DataFrame: Podzbiór `df_songs` zawierający najlepszych kandydatów,
        z dodanymi kolumnami 'tag_score' oraz 'score'.
    """
    # bierzemy tylko te, które w ogóle coś dopasowały
    pos_mask = scores_songs_tag > 0.0
    if not np.any(pos_mask):
        return df_songs.iloc[0:0].copy()

    pos_idxs = np.where(pos_mask)[0]
    # sortujemy wszystkie dopasowane po score malejąco
    pos_idxs = pos_idxs[np.argsort(-scores_songs_tag[pos_idxs])]

    # jeśli i tak mniej niż n_candidates → bierzemy wszystkie
    if len(pos_idxs) <= n_candidates:
        final_idxs = pos_idxs
    else:
        # score na granicy top-n
        boundary_idx = n_candidates - 1
        boundary_score = float(scores_songs_tag[pos_idxs[boundary_idx]])

        # rozszerzamy cut tak długo, jak kolejne mają ten sam (w praktyce: bardzo podobny) score
        cut = n_candidates
        while (
            cut < len(pos_idxs)
            and abs(float(scores_songs_tag[pos_idxs[cut]]) - boundary_score) <= flat_delta
        ):
            cut += 1

        final_idxs = pos_idxs[:cut]

    candidates = df_songs.iloc[final_idxs].copy()
    candidates["tag_score"] = scores_songs_tag[final_idxs]
    candidates["score"] = candidates["tag_score"]  # spójność z resztą pipeline'u
    return candidates

In [ ]:
def calculate_audio_match(candidates_df, audio_criteria):
    """
    Oblicza stopień dopasowania kandydatów do zadanych kryteriów audio (np. energy, danceability).
    Stosuje podejście iloczynowe: ocena końcowa to iloczyn dopasowań dla poszczególnych cech.

    Logika:
    1. Dla każdego kryterium obliczany jest dystans: |wartość_utworu - wartość_docelowa|.
    2. Dystans zamieniany jest na podobieństwo: 1.0 - dystans.
    3. Wyniki cząstkowe są mnożone (utwór musi być bliski wszystkim kryteriom naraz).

    Args:
        candidates_df (pd.DataFrame): DataFrame z utworami. Musi zawierać kolumny odpowiadające
            nazwom cech w audio_criteria (np. 'energy', 'tempo').
        audio_criteria (List[Tuple[str, Dict[str, float]]]): Lista krotek z kryteriami.
            Format: [('energy', {'value': 0.8, ...}), ('danceability', {'value': 0.5})].

    Returns:
        np.ndarray: Wektor o długości równej liczbie wierszy w candidates_df,
        zawierający wartości zmiennoprzecinkowe z zakresu 0.0 - 1.0.
    """
    if candidates_df.empty:
        return np.array([])

    if not audio_criteria:
        return np.ones(len(candidates_df))

    total_audio_score = np.ones(len(candidates_df))

    for feature_name, criteria in audio_criteria:
        val_data = criteria['value']
        target_min, target_max = val_data

        if feature_name not in candidates_df.columns:
            continue

        song_values = candidates_df[feature_name].to_numpy()

        dist_below = np.maximum(0, target_min - song_values)
        dist_above = np.maximum(0, song_values - target_max)
        total_dist = dist_below + dist_above

        sim = np.clip(1.0 - total_dist, 0.0, 1.0)

        total_audio_score *= sim

    return total_audio_score

Fuzja - łączymy wyniki audio i tagów i robimy scoring

In [ ]:
def merge_tag_and_audio_scores(df, audio_weight=0.3, use_tags=True):
    df = df.copy()
    w = audio_weight if use_tags else 1.0

    df['score'] = (df.get('tag_score', 0) * (1 - w)) + (df.get('audio_score', 0) * w)

    for col in ['tag_score', 'audio_score']:
        if col not in df: df[col] = 0.0

    return df.sort_values('score', ascending=False)

In [ ]:
candidades_df = retrieve_candidates_tags(
    scores_songs_tag,
    n_candidates=RETRIEVAL_CONFIG['n_candidates'],
    flat_delta=RETRIEVAL_CONFIG['flat_delta']
)
candidades_df

,track_id,name,artist,spotify_preview_url,tags,genre,year,duration_ms,danceability,energy,...,album_images,spotify_id,n_tempo,n_loudness,tags_list,tags_count,tags_embedding,tag_count,tag_score,score
41563,TRXXCYI128F425E9AC,Yours Forever,Dara Maclean,https://p.scdn.co/mp3-preview/c8e6524ed7d56927...,"rock, alternative, indie, pop, metal, classic_...",None,2011,221920,0.470,0.394,...,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",3kgNO3OYninekOkLIWYlF5,0.544093,0.822020,"[80s, 90s, alternative, beautiful, british, cl...",28,"[0.008944714441895485, 0.05254964902997017, -0...",28,1.000000,1.000000
41570,TRMBNHW128F425E820,Blameless,Dara Maclean,https://p.scdn.co/mp3-preview/a43a7e7c56665405...,"rock, alternative, indie, pop, metal, classic_...",None,2014,234348,0.355,0.677,...,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",3hJf9RCc6jLId4U99rSTa2,0.669679,0.828007,"[80s, 90s, alternative, beautiful, british, cl...",29,"[0.012565230950713158, 0.05316093564033508, -0...",29,1.000000,1.000000
41507,TRRGYHW128F9323586,In The Woods,NORA&LEO,https://p.scdn.co/mp3-preview/153890cb4d271734...,"rock, electronic, alternative, indie, pop, fem...",Rock,2010,99744,0.493,0.555,...,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",1YW63oNKffmqHHm7oGxtUZ,0.439565,0.828399,"[80s, acoustic, alternative, alternative rock,...",39,"[0.0047281477600336075, 0.042138393968343735, ...",39,1.000000,1.000000
41536,TRPEGXI128F9327AB5,Suitcases,Dara Maclean,https://p.scdn.co/mp3-preview/9faff44322cb1ab3...,"rock, alternative, indie, pop, metal, classic_...",None,2012,214226,0.551,0.942,...,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",03f7wC7s2B68iPJD09O4o3,0.678507,0.885516,"[80s, 90s, alternative, beautiful, british, cl...",29,"[0.012565230950713158, 0.05316093564033508, -0...",29,1.000000,1.000000
37025,TRMBLYH128F92F9F8A,Tropical,Pompeya,https://p.scdn.co/mp3-preview/edecde5ec6c5e30a...,"indie, indie_rock, new_wave, russian",None,2011,374935,0.618,0.957,...,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",75YN77Tl0e1QgojDafhSDp,0.523263,0.843138,"[indie, indie rock, new wave, rock, russian]",4,"[0.011679689399898052, 0.04011470451951027, -0...",4,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37021,TRDPLZA12903CC3308,Distant,Ocean Jet,https://p.scdn.co/mp3-preview/a600eb9e551a113e...,"indie, russian",None,2014,229000,0.680,0.961,...,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",6Cq1uN23Oijp7QWLBfd9dF,0.410251,0.886883,"[indie, russian]",2,"[0.02164691872894764, 0.02858036570250988, -0....",2,0.481061,0.481061
36990,TRQGVUF12903CA6FE5,Neon Love,Tesla Boy,https://p.scdn.co/mp3-preview/d884c2f31d5450f9...,"electronic, synthpop, russian",None,2012,432000,0.519,0.886,...,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",3zCd0B3Usbp9JCUMrFwatc,0.376764,0.825100,"[electronic, pop, russian, synthpop]",3,"[0.010082477703690529, 0.019515978172421455, -...",3,0.481061,0.481061
36991,TREEQWK12903CA5A80,Runaway Man,Tesla Boy,https://p.scdn.co/mp3-preview/aea169e4f932095d...,"electronic, electro, synthpop, russian",None,2009,359925,0.580,0.940,...,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",3frNLuKDpuMgWlR6KXCElb,0.665548,0.860768,"[electro, electronic, pop, russian, synthpop]",4,"[0.009125862270593643, 0.022189870476722717, 0...",4,0.481061,0.481061
36992,TRZBHAZ12903CC71ED,Cthulhu,The Acacia Strain,https://p.scdn.co/mp3-preview/4c2e5ecfef8e93a3...,russian,None,2008,207360,0.311,0.971,...,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",18YiD1V1Jcj7IlLPKjg4Vm,0.523330,0.867525,[russian],1,"[0.023847544565796852, 0.005697064101696014, 0...",1,0.481061,0.481061


In [ ]:
has_tags = not candidades_df.empty

# Fallback
if not has_tags and criteria_audio:
    print("Brak wyników z tagów.")
    candidades_df = df_songs.copy()
elif has_tags:
    print(f"Znaleziono {len(candidades_df)} kandydatów po tagach.")
else:
    print("Pusto (brak tagów i brak audio).")

Znaleziono 17442 kandydatów po tagach.


In [ ]:
print(picked_prompt)

muzyka hip hop


In [ ]:
if not candidades_df.empty and criteria_audio:
    candidades_df['audio_score'] = calculate_audio_match(candidades_df, criteria_audio)

In [ ]:
audio_match_vector = calculate_audio_match(candidades_df, criteria_audio)
candidades_df['audio_score'] = audio_match_vector
candidades_df[['artist', 'name', 'audio_score']].sample(3)

,artist,name,audio_score
10192,David Bowie,Wild Is the Wind,0.604218
26028,The Octopus Project,All of the Champs That Ever Lived,0.877990
12354,Sufjan Stevens,Romulus,0.343485


In [ ]:
picked_prompt

'muzyka hip hop'

In [ ]:
candidades_merged_score_df = merge_tag_and_audio_scores(candidades_df, audio_weight=0.6, use_tags=has_tags)
candidades_merged_score_df[['artist', 'name', 'tag_score', 'audio_score', 'score']].head(20)

,artist,name,tag_score,audio_score,score
41536,Dara Maclean,Suitcases,1.000000,0.978507,0.987104
1777,Rammstein,Moskau,1.000000,0.913049,0.947829
37027,That Poppy,Altar,1.000000,0.830231,0.898139
37025,Pompeya,Tropical,1.000000,0.823263,0.893958
25598,GG Allin,"Drink, fight and fuck",1.000000,0.761680,0.857008
3409,Dua Lipa,Hotter Than Hell,1.000000,0.760922,0.856553
41570,Dara Maclean,Blameless,1.000000,0.727099,0.836260
11719,Chelsea Wolfe,The Warden,1.000000,0.725196,0.835118
16358,Chelsea Wolfe,Scrape,1.000000,0.707000,0.824200
15892,Chelsea Wolfe,Grey Days,1.000000,0.694216,0.816529


Dzielimy zestaw kandydatów (piosenek) na trzy poziomy jakości (tzw. tiery) w zależności od wartości dopasowania (score).

Celem tej funkcji jest rozdzielenie wyników wyszukiwania według jakości dopasowania,
tak aby późniejsze etapy
mogły w różny sposób traktować utwory bardzo podobne, średnio podobne i słabe.

Utwory są przypisywane do trzech grup:
  * Tier A — wysokie dopasowanie (`score` >= `t_high`)
  * Tier B — średnie dopasowanie (`t_mid` <= `score` < `t_high`)
  * Tier C — niskie dopasowanie (`score` < `t_mid`)

In [ ]:
def tier_by_score(candidates: pd.DataFrame, t_high: float, t_mid: float):
    """
    Dzieli kandydatów na trzy poziomy jakości (tiery) w zależności od wartości dopasowania (score).

    - Tier A: score >= t_high - bardzo dobre dopasowanie
    - Tier B: t_mid <= score < t_high - średnie dopasowanie
    - Tier C: score < t_mid - słabe dopasowanie

    Args:
        candidates (pd.DataFrame): Dane z kolumną 'score'.
        t_high (float): Próg dla wysokiego dopasowania.
        t_mid (float): Próg dla średniego dopasowania.

    Returns:
        tuple: (tier_a, tier_b, tier_c) — trzy DataFramey z podziałem wg jakości.
    """
    tier_a = candidates[candidates["score"] >= t_high].copy()
    tier_b = candidates[(candidates["score"] < t_high) & (candidates["score"] >= t_mid)].copy()
    tier_c = candidates[candidates["score"] < t_mid].copy()
    return tier_a, tier_b, tier_c

In [ ]:
def calculate_dynamic_thresholds(candidates_df, high_threshold=0.75, mid_threshold=0.5):
    """
    Wylicza adaptacyjne progi punktowe (t_high, t_mid) służące do podziału wyników na Tiery (np. A i B).

    Logika łączy podejście względne (relative) z bezwzględnym (absolute):
    1. Podejście względne: Próg dostosowuje się do najlepszego wyniku w danej puli (max_score).
       Jeśli mamy świetne dopasowania (np. 0.95), próg Tier A rośnie (np. do 0.85), aby wybrać tylko "śmietankę".
    2. Podejście bezwzględne (Quality Gate): Próg nigdy nie spadnie poniżej ustalonego minimum (high_threshold).
       Jeśli najlepszy wynik to tylko 0.6, a high_threshold to 0.75, to t_high wyniesie 0.75.
       W efekcie Tier A pozostanie pusty, co jest pożądanym zachowaniem (nie pokazujemy słabych wyników jako "świetne").

    Args:
        candidates_df (pd.DataFrame): DataFrame z kandydatami, musi zawierać kolumnę 'score'.
        high_threshold (float, optional): Minimalna wartość bezwzględna dla Tier A. Domyślnie 0.75.
        mid_threshold (float, optional): Minimalna wartość bezwzględna dla Tier B. Domyślnie 0.5.

    Returns:
        Tuple[float, float]: Krotka zawierająca wyliczone progi (t_high, t_mid).
    """
    if candidates_df.empty:
        return 0.0, 0.0

    max_score = candidates_df['score'].max()

    # logika adaptacyjna
    t_high = max(high_threshold, max_score - 0.1)

    # tier B
    t_mid = max(mid_threshold, max_score - 0.2)

    return t_high, t_mid

In [ ]:
t_high, t_mid = calculate_dynamic_thresholds(
    candidades_merged_score_df,
    high_threshold=WORKSET_CONFIG['min_absolute_high'],
    mid_threshold=WORKSET_CONFIG['min_absolute_mid']
)
t_high, t_mid

(0.887104382360138, 0.787104382360138)

In [ ]:
tier_a, tier_b, tier_c = tier_by_score(
    candidades_merged_score_df,
    t_high,
    t_mid
)
print(len(tier_a), len(tier_b), len(tier_c))

4 291 17147


 Budowanie zbioru, z którego będziemy losować/wybierać utwory na podstawie dalszych cech (np. popularności).

Funkcja ma na celu stworzenie wystarczająco dużej i jakościowo sensownej puli kandydatów, z którymi będzie dalej pracować algorytm rekomendacji. Nie wybiera jeszcze finalnych piosenek, przygotowuje "roboczy" zestaw do późniejszego ważenia, filtrowania i losowania.


- min_final - mininalna wielkość zbioru, z którego potem będziemy czerpać.
- A - duży score
- B - średni
- C - niski

Logika:
* jeśli A >= min_final -> używamy tylko A
* elif A+B >= min_final -> używamy A + B
* else -> A + B + kawałek C

In [ ]:
def build_working_set(
    tier_a: pd.DataFrame,
    tier_b: pd.DataFrame,
    tier_c: pd.DataFrame,
    target_pool_size: int,
    min_required_size: int,
    popularity_rescue_ratio: float = 0.2
) -> pd.DataFrame:
    """
    Buduje zbiór roboczy.
    Ważne: Przy dobieraniu z Tier B, dba o to, aby nie wziąć tylko niszowych utworów z dobrym score,
    ale też "uratować" popularne utwory, które mają przyzwoity score (są w Tier B).
    Args:
        tier_a (pd.DataFrame): Utwory o wysokim dopasowaniu (Hity jakościowe).
                               Bierzemy je zawsze w całości.
        tier_b (pd.DataFrame): Utwory o średnim dopasowaniu (Dobre, ale nie idealne).
                               Służą do uzupełnienia puli do `target_pool_size`.
        tier_c (pd.DataFrame): Utwory o niskim dopasowaniu (Słabe).
                               Używane tylko awaryjnie, jeśli nie osiągniemy `min_required_size`.
        target_pool_size (int): Docelowa wielkość zbioru roboczego (np. 50-100).
                                Tyle kandydatów chcemy dać algorytmowi losującemu,
                                aby miał swobodę w dobieraniu High/Mid/Low popularity.
        min_required_size (int): ABSOLUTNE MINIMUM. Tyle piosenek ma mieć finalna playlista.
                                 Jeśli A+B to za mało, dobierzemy stąd resztę z Tier C.
                                 Zalecane: tyle, ile user zażądał (np. 15).
        popularity_rescue_ratio (float): Jaka część slotów z Tieru B ma być zarezerwowana
                                         dla utworów NAJPOPULARNIEJSZYCH (zamiast tych z najlepszym score).
                                         Np. 0.3 oznacza, że 30% dobieranych utworów z B to hity,
                                         a 70% to najlepiej pasujące niszowe.
    """

    working_parts = [tier_a]
    current_count = len(tier_a)

    if current_count < target_pool_size:
        needed = target_pool_size - current_count

        if len(tier_b) <= needed:
            working_parts.append(tier_b)
            current_count += len(tier_b)
        else:
            n_pop = int(needed * popularity_rescue_ratio)
            n_score = needed - n_pop

            b_pop_rescued = tier_b.sort_values("popularity", ascending=False).head(n_pop)
            remaining_b = tier_b.drop(b_pop_rescued.index)
            b_score_top = remaining_b.sort_values("score", ascending=False).head(n_score)

            working_parts.extend([b_pop_rescued, b_score_top])
            current_count += (len(b_pop_rescued) + len(b_score_top))

    if current_count < min_required_size:
        needed = min_required_size - current_count
        part_c = tier_c.sort_values("score", ascending=False).head(needed)
        working_parts.append(part_c)

    if not working_parts:
        return pd.DataFrame(columns=tier_a.columns)

    working = pd.concat(working_parts, ignore_index=True)
    subset_cols = []

    if 'id' in working.columns:
        subset_cols = ['id']
    elif 'name' in working.columns and 'artist' in working.columns:
        subset_cols = ['name', 'artist']

    if subset_cols:
        working = working.drop_duplicates(subset=subset_cols)
    else:
        pass

    working = working.sort_values("score", ascending=False).reset_index(drop=True)

    return working

In [ ]:
working = build_working_set(
    tier_a, tier_b, tier_c,
    target_pool_size=WORKSET_CONFIG['target_pool_size'],
    min_required_size=WORKSET_CONFIG['min_required_size'],
    popularity_rescue_ratio=WORKSET_CONFIG['popularity_rescue_ratio']
)
# working
working["score"].value_counts()

,count
score,
0.807576,75
0.801576,2
0.898139,1
0.947829,1
0.857008,1
0.856553,1
0.836260,1
0.893958,1
0.987104,1


Podział kandydatów wg popularności - chcemy uzyskać część popularnych utworów, część niszowych.

In [ ]:
def bucket_by_popularity(working: pd.DataFrame, p_high: int, p_mid: int):
    """
    Dzieli utwory na trzy grupy (bucket'y) według ich popularności.

    - High:  popularity >= p_high
    - Mid:   p_mid <= popularity < p_high
    - Low:   popularity < p_mid

    Args:
        working (pd.DataFrame): Zbiór roboczy utworów z kolumną 'popularity'.
        p_high (int): Próg dla wysokiej popularności.
        p_mid (int): Próg dla średniej popularności.

    Returns:
        tuple: (pop_high, pop_mid, pop_low) — trzy DataFrame'y z podziałem wg popularności.
    """


    pop_high = working[working["popularity"] >= p_high].copy()
    pop_mid = working[(working["popularity"] < p_high) & (working["popularity"] >= p_mid)].copy()
    pop_low = working[working["popularity"] < p_mid].copy()
    return pop_high, pop_mid, pop_low

In [ ]:
pop_high, pop_mid, pop_low = bucket_by_popularity(working, POPULARITY_CONFIG['p_high'], POPULARITY_CONFIG['p_mid'])
pop_high

,track_id,name,artist,spotify_preview_url,tags,genre,year,duration_ms,danceability,energy,...,spotify_id,n_tempo,n_loudness,tags_list,tags_count,tags_embedding,tag_count,tag_score,score,audio_score
10,TRXGAXB128F4272C8F,Thunder,Imagine Dragons,https://p.scdn.co/mp3-preview/07e14c4e82117429...,"rock, alternative, indie, pop, alternative_roc...",None,2017,187761,0.608,0.841,...,1zB4vmk8tFRmM9UULNzbLB,0.703108,0.883096,"[alternative, alternative rock, american, indi...",12,"[0.006795514840632677, 0.02580062672495842, -0...",12,0.518939,0.807576,1.000000
11,TRXDBRF128F930DC7D,Bad Moon Rising,Creedence Clearwater Revival,https://p.scdn.co/mp3-preview/9c758b0f4d7e6377...,"rock, classic_rock, 60s, oldies",None,2010,155271,0.512,0.820,...,20OFwXhEXf12DzwXmaV7fj,0.775872,0.825744,"[60s, classic rock, oldies, rock]",4,"[0.010572127997875214, 0.01460581086575985, -0...",4,0.518939,0.807576,1.000000
14,TRXKWTI128F93452C0,This Is the Life,Amy Macdonald,https://p.scdn.co/mp3-preview/ec438f2fe9cdc995...,"rock, indie, pop, female_vocalists, folk, sing...",None,2014,184093,0.514,0.887,...,2T3jgoAMbBXiMMD8pSc6QL,0.795789,0.853650,"[acoustic, country, female vocalists, folk, in...",8,"[-0.002751805353909731, 0.03456230089068413, -...",8,0.518939,0.807576,1.000000
15,TRAAKCH128F42609D0,Footloose,Kenny Loggins,https://p.scdn.co/mp3-preview/76d526694040f175...,"rock, pop, dance, 80s, soundtrack",None,2011,228440,0.537,0.938,...,6W2VbtvMrDXm5vYeB7amkO,0.727818,0.875664,"[80s, dance, pop, rock, soundtrack]",5,"[0.020170796662569046, 0.025928236544132233, -...",5,0.518939,0.807576,1.000000
85,TRKKFQD128F1482B82,All the Things She Said,t.A.T.u.,https://p.scdn.co/mp3-preview/0895b460a675006e...,"rock, electronic, pop, female_vocalists, dance...",None,2014,213333,0.601,0.799,...,4bJygwUKrRgq1stlNXcgMg,0.376739,0.837073,"[00s, dance, electronic, female vocalists, pop...",7,"[0.006092104595154524, 0.014883558265864849, -...",7,1.000000,0.805637,0.676062
86,TRYNOSN128F42A2B82,The Anthem,Good Charlotte,https://p.scdn.co/mp3-preview/37bf3c2518b7d23d...,"rock, alternative, pop, alternative_rock, punk...",Pop,2002,175093,0.494,0.939,...,0BRHnOFm6sjxN1i9LJrUDu,0.744055,0.893639,"[alternative, alternative rock, pop, pop rock,...",7,"[-0.008033597841858864, 0.022734323516488075, ...",7,0.518939,0.803976,0.994000
87,TRIDQPM128F932BEB1,Headstrong,Trapt,https://p.scdn.co/mp3-preview/08bf3158a4f3982b...,"rock, alternative, alternative_rock, hard_rock...",None,2002,285569,0.492,0.896,...,1AEYT6VxrxXPMoQUxsY0E4,0.774319,0.851340,"[alternative, alternative rock, hard rock, met...",5,"[-0.00401513883844018, 0.034884486347436905, -...",5,0.518939,0.802776,0.992000
88,TRDDCWY128F9344362,Me and My Broken Heart,Rixton,https://p.scdn.co/mp3-preview/7c2a3ca296646045...,"pop, british, male_vocalists, love, pop_rock",None,2014,192746,0.526,0.791,...,1oew3nFNY3vMacJAsvry0S,0.728508,0.869693,"[british, love, male vocalists, pop, pop rock,...",5,"[0.01600559614598751, 0.023921430110931396, -0...",5,0.518939,0.802176,0.991000
89,TRXKRZI128F932C312,Blitzkrieg Bop,Ramones,https://p.scdn.co/mp3-preview/9ce6132e4c21f483...,"rock, classic_rock, punk, punk_rock, 70s",Rock,2011,134360,0.532,0.790,...,4KcH1ZRV2W1q7Flq0QqC76,0.737345,0.852330,"[70s, classic rock, punk, punk rock, rock]",5,"[-0.011295706033706665, 0.02174096368253231, -...",5,0.518939,0.801576,0.990000
90,TRZGABY128F1462E00,Ocean Avenue,Yellowcard,https://p.scdn.co/mp3-preview/b2b5d20865277b4a...,"rock, alternative, pop, alternative_rock, punk...",Rock,2011,198973,0.490,0.912,...,23oxJmDc1V9uLUSmN2LIvx,0.727487,0.879796,"[alternative, alternative rock, emo, love, pop...",8,"[-0.00753061193972826, 0.030665148049592972, -...",8,0.518939,0.801576,0.990000


In [ ]:
def weighted_sample(df: pd.DataFrame, k: int, alpha: float):
    """
    Losuje `k` wierszy z DataFrame bez zwracania, faworyzując utwory z wyższym 'score'.
    Prawdopodobieństwo wylosowania utworu jest proporcjonalne do: (score ^ alpha).

    Rola parametru alpha (temperatura losowania):
    - alpha > 1 (np. 3.0): Strategia "Greedy" (chciwa). Bardzo mocno faworyzuje utwory z najwyższym wynikiem.
      Szansa na wylosowanie słabszych utworów drastycznie maleje.
    - alpha = 1.0: Prawdopodobieństwo liniowo zależne od wyniku.
    - alpha < 1 (np. 0.5): Spłaszczenie różnic. Słabsze utwory mają większą szansę zaistnieć.
    - alpha = 0: Losowanie jednostajne (każdy utwór ma taką samą szansę, ignorujemy score).

    Args:
        df (pd.DataFrame): Tabela z kandydatami (musi zawierać kolumnę 'score').
        k (int): Liczba utworów do wylosowania.
        alpha (float): Współczynnik skalowania wag (wykładnik potęgi).

    Returns:
        pd.DataFrame: Podzbiór `df` zawierający `k` wylosowanych wierszy.
    """
    if k <= 0 or len(df) == 0:
        return df.iloc[0:0]

    k = min(k, len(df))

    scores = df["score"].to_numpy(dtype=np.float32)
    scores = np.clip(scores, 1e-9, None)
    w = np.power(scores, alpha)
    w_sum = w.sum()
    if w_sum <= 0:
        w = np.full_like(w, 1.0 / len(w), dtype=np.float32)
    else:
        w /= w_sum

    idx = np.random.choice(len(df), size=k, replace=False, p=w)
    return df.iloc[idx]

In [ ]:
def sample_final_songs(
    working: pd.DataFrame,
    popularity_cfg: dict,
    sampling_cfg: dict,
) -> pd.DataFrame:
    """
    Dokonuje finalnego wyboru utworów do playlisty z przygotowanej "puli roboczej" (working set).
    Stosuje zaawansowaną strategię mieszania popularności (Popularity Mix) oraz ważonego losowania (Weighted Sampling).
    Gdy pożądanych utworów jest mała (5), to wykonuje losowanie bez podziału na buckety.

    Strategia:
    1. Forced Popular (Kotwice): Gwarantuje obecność znanych hitów (np. 2-3 utwory), aby użytkownik czuł się bezpiecznie.
       Stosuje losowanie ważone, aby nie grać w kółko tych samych 5 najpopularniejszych piosenek
    2. Popularity Buckets: Dzieli resztę miejsc wg proporcji (np. 40% High, 40% Mid, 20% Low/Discovery).
    3. Weighted Audio/Tag Score: Wewnątrz każdego bucketa utwory są losowane z prawdopodobieństwem zależnym od ich dopasowania
       do zapytania (score). Dzięki temu playlista trzyma klimat (audio/tagi), ale jest różnorodna.

    Args:
        working (pd.DataFrame): Pula kandydatów (zbiór roboczy) z kolumnami 'score' i 'popularity'.
        popularity_cfg (Dict[str, Any]): Konfiguracja popularności.
            - 'p_high', 'p_mid' (int): Progi punktowe podziału na buckety.
            - 'mix' (dict): Proporcje (np. {'high': 0.4, 'mid': 0.4, 'low': 0.2}).
            - 'forced_popular' (int): Liczba gwarantowanych hitów na start.
            - 'forced_popular_min' (int): Minimalna popularność, by utwór uznano za "hit" do sekcji forced.
        sampling_cfg (Dict[str, Any]): Konfiguracja losowania.
            - 'final_n' (int): Docelowa długość playlisty.
            - 'alpha' (float): "Temperatura" losowania. Im wyższa, tym bardziej faworyzujemy utwory z wysokim score.
            - 'shuffle' (bool): Czy przetasować finalną listę (True) czy zostawić posortowaną po score (False/Debug).

    Returns:
        pd.DataFrame: Gotowa playlista o długości `final_n` (lub mniejszej, jeśli brakło kandydatów).
    """

    if len(working) == 0:
        return working.iloc[0:0].copy()

    final_n = sampling_cfg.get("final_n", 15)
    alpha   = sampling_cfg.get("alpha", 2.0)

    # mało piosenek - losowe
    if final_n < 5:
        return weighted_sample(working, final_n, alpha)

    p_high = popularity_cfg.get("p_high", 70)
    p_mid  = popularity_cfg.get("p_mid", 35)
    mix    = popularity_cfg.get("mix", {"high": 0.4, "mid": 0.35, "low": 0.25})

    forced_popular         = popularity_cfg.get("forced_popular", 0)
    forced_popular_min     = popularity_cfg.get("forced_popular_min", p_high)

    # bucketowanie po popularity
    pop_high, pop_mid, pop_low = bucket_by_popularity(working, p_high=p_high, p_mid=p_mid)

    final_parts = []

    # Forced popular
    forced_pool = working[working["popularity"] >= forced_popular_min].copy()
    forced_taken = weighted_sample(forced_pool, forced_popular, alpha)
    final_parts.append(forced_taken)

    used_idx = set(forced_taken.index)
    pop_high = pop_high[~pop_high.index.isin(used_idx)]
    pop_mid  = pop_mid[~pop_mid.index.isin(used_idx)]
    pop_low  = pop_low[~pop_low.index.isin(used_idx)]

    # Obliczamy ile slotów zostało do wypełnienia
    n_forced = len(forced_taken)
    remaining = max(0, final_n - n_forced)

    if remaining == 0:
        combined = pd.concat(final_parts, ignore_index=False)
        return combined.sort_values("score", ascending=False).reset_index(drop=True)

    # Wyliczamy cele dla bucketów
    target_high = int(round(remaining * mix.get("high", 0.0)))
    target_mid  = int(round(remaining * mix.get("mid",  0.0)))
    target_low  = remaining - target_high - target_mid

    target_high = min(target_high, len(pop_high))
    target_mid  = min(target_mid, len(pop_mid))
    target_low  = min(target_low, len(pop_low))

    # losowanie z bucketów
    sampled_high = weighted_sample(pop_high, target_high, alpha)
    sampled_mid  = weighted_sample(pop_mid,  target_mid,  alpha)
    sampled_low  = weighted_sample(pop_low,  target_low,  alpha)

    final_parts.extend([sampled_high, sampled_mid, sampled_low])

    combined = pd.concat(final_parts, ignore_index=False)

    # fill gaps
    if len(combined) < final_n:
        missing = final_n - len(combined)
        used_idx = set(combined.index)

        remaining_pool = working[~working.index.isin(used_idx)]
        extra = weighted_sample(remaining_pool, missing, alpha)

        combined = pd.concat([combined, extra], ignore_index=False)

    # finalne tasowanie
    do_shuffle = sampling_cfg.get("shuffle", True)
    if do_shuffle:
        combined = combined.sample(frac=1).reset_index(drop=True)
    else:
        # for debugging
        combined = combined.sort_values("score", ascending=False).reset_index(drop=True)

    return combined


In [ ]:
picked_prompt

'muzyka hip hop'

In [ ]:
sample_final_songs(
    working,
    POPULARITY_CONFIG,
    SAMPLING_CONFIG
)[["name", "artist", "tags_list", "popularity", "score", "n_tempo", "energy", "valence", "instrumentalness", "n_loudness"]]

,name,artist,tags_list,popularity,score,n_tempo,energy,valence,instrumentalness,n_loudness
0,Thunder,Imagine Dragons,"[alternative, alternative rock, american, indi...",85.0,0.807576,0.703108,0.841,0.274,0.057300,0.883096
1,Two Birds,Regina Spektor,"[alternative, american, chill, experimental, f...",72.0,0.791332,0.631742,0.500,0.449,0.001080,0.803227
2,Deceptacon,Le Tigre,"[alternative, dance, electro, electronic, fema...",79.0,0.790619,0.671739,0.864,0.625,0.005240,0.889633
3,Charley's Girl,Lou Reed,"[70s, classic rock, oldies, rock, singer songw...",50.0,0.807576,0.723996,0.820,0.897,0.150000,0.814824
4,This Is the New Shit,Marilyn Manson,"[alternative, alternative rock, gothic, hard r...",63.0,0.807576,0.745516,0.873,0.616,0.005050,0.846894
5,Headstrong,Trapt,"[alternative, alternative rock, hard rock, met...",73.0,0.802776,0.774319,0.896,0.534,0.000003,0.851340
6,Speak Easy,311,"[alternative, rock, ska]",31.0,0.807576,0.730878,0.867,0.591,0.000013,0.836539
7,Para-noir,Marilyn Manson,"[industrial, metal, rock]",34.0,0.807576,0.753800,0.904,0.304,0.282000,0.840561
8,Doperide,Saliva,"[hard rock, metal, nu metal, rock]",46.0,0.807576,0.795395,0.897,0.705,0.088900,0.885689
9,Hey Baby,No Doubt,"[00s, 90s, alternative, alternative rock, danc...",59.0,0.807576,0.784307,0.852,0.760,0.000000,0.871311


In [ ]:
def generate_playlist(prompt: str, n_songs: int = 15) -> pd.DataFrame:
    """
    Główna funkcja pipeline'u rekomendacji muzycznych.
    Przetwarza prompt użytkownika i generuje playlistę dopasowanych utworów.

    Args:
        prompt (str): Zapytanie użytkownika opisujące pożądaną muzykę.
        n_songs (int): Liczba utworów w finalnej playliście. Domyślnie 15.

    Returns:
        pd.DataFrame: DataFrame z finalną playlistą.
    """

    print("=" * 80)
    print("PLAYLIST GENERATION PIPELINE")
    print("  Input prompt: '{}'".format(prompt))
    print("  Requested tracks: {}".format(n_songs))
    print("=" * 80)

    sampling_config = SAMPLING_CONFIG.copy()
    sampling_config['final_n'] = n_songs

    try:
        lang_code = detect(picked_prompt)
    except:
        lang_code = 'pl'

    if lang_code == 'en':
        current_nlp = nlp_en
        current_matcher = matcher_en
        lang_txt = 'EN'
    else:
        current_nlp = nlp_pl
        current_matcher = matcher_pl
        lang_txt = 'PL'

    # PHASE 1: PHRASE EXTRACTION
    print("\n[PHASE 1] PHRASE EXTRACTION")
    print("-" * 40)

    relevant_phrases = extract_relevant_phrases(prompt, current_nlp, current_matcher)

    if not relevant_phrases:
        print("[ERROR] No relevant phrases found. Aborting.")
        return pd.DataFrame()

    print("[OK] Extracted {} phrases: {}".format(len(relevant_phrases), relevant_phrases))

    # PHASE 2: CLASSIFICATION & ROUTING
    print("\n")
    print("=" * 80)
    print("PHASE 2: CLASSIFICATION & ROUTING")
    print("=" * 80)

    classified_phrases = classify_phrases_with_gliner(
        prompt,
        relevant_phrases,
        model_gliner,
        current_nlp,
        threshold=EXTRACTION_CONFIG['gliner_threshold']
    )

    print("\n{:<30} {:<10} {:<25}".format("PHRASE", "ROUTE", "CATEGORY"))
    print("-" * 65)
    for item in classified_phrases:
        print("{:<30} {:<10} {:<25}".format(
            item['phrase'],
            item['route'],
            item['category']
        ))
    print("-" * 65)

    e5_queries_separated = prepare_queries_for_e5_separated(classified_phrases, prompt)

    tags_queries = e5_queries_separated['TAGS']
    audio_queries = e5_queries_separated['AUDIO']

    print("\nROUTING RESULT:")
    print("  TAGS  -> {}".format(tags_queries if tags_queries else "(none)"))
    print("  AUDIO -> {}".format(audio_queries if audio_queries else "(none)"))
    print("  Language: {}".format(lang_code.upper()))
    print("=" * 80)

    # PHASE 3: AUDIO FEATURE MAPPING
    print("\n[PHASE 3] AUDIO FEATURE MAPPING")
    print("-" * 40)

    criteria_audio = []
    if audio_queries:
        criteria_audio = phrases_to_features(audio_queries, SEARCH_INDICES, lang_code=lang_code)
        if criteria_audio:
            print("\n[OK] Mapped {} audio criteria:".format(len(criteria_audio)))
            for feat, data in criteria_audio:
                print("     {}: {} (conf: {:.3f})".format(feat, data['value'], data['confidence']))
    else:
        print("[INFO] No audio phrases to map")

    # PHASE 4: TAG MAPPING
    print("\n[PHASE 4] TAG MAPPING")
    print("-" * 40)

    criteria_tags = {}
    if tags_queries:
        criteria_tags = map_phrases_to_tags(
            tags_queries,
            model=model_e5,
            tag_vecs=TAG_VECS,
            tags_list=TAGS,
            threshold_strict=EXTRACTION_CONFIG['tag_similarity_threshold'],
            threshold_lenient=EXTRACTION_CONFIG["tag_similarity_threshold_lenient"]
        )
        if criteria_tags:
            print("\n[OK] Mapped {} tags".format(len(criteria_tags)))
    else:
        print("[INFO] No tag phrases to map")

    # PHASE 5: TAG SCORING
    print("\n[PHASE 5] TAG SCORING")
    print("-" * 40)

    query_tag_weights = get_query_tag_weights(criteria_tags)

    if query_tag_weights:
        print("Tag weights (normalized):")
        for tag, weight in sorted(query_tag_weights.items(), key=lambda x: x[1], reverse=True):
            print("     '{}': {:.3f}".format(tag, weight))
    else:
        print("[INFO] No tags for scoring")

    scores_songs_tag = score_songs_by_tags(
        query_tag_weights=query_tag_weights,
        inv_index=INV_INDEX,
        n_songs=len(df_songs),
        idf_map=IDF_MAP if SCORING_CONFIG['use_idf'] else None,
        use_idf=SCORING_CONFIG['use_idf'],
        query_pow=SCORING_CONFIG['query_pow']
    )

    positive_scores = np.sum(scores_songs_tag > 0)
    max_tag_score = np.max(scores_songs_tag) if positive_scores > 0 else 0
    print("[OK] Tracks with positive tag score: {}, max: {:.3f}".format(positive_scores, max_tag_score))

    # PHASE 6: CANDIDATE RETRIEVAL
    print("\n[PHASE 6] CANDIDATE RETRIEVAL")
    print("-" * 40)

    candidates_df = retrieve_candidates_tags(
        scores_songs_tag,
        n_candidates=RETRIEVAL_CONFIG['n_candidates'],
        flat_delta=RETRIEVAL_CONFIG['flat_delta']
    )

    has_tags = not candidates_df.empty

    if not has_tags and criteria_audio:
        print("[WARN] No tag results - using full database with audio filtering")
        candidates_df = df_songs.copy()
        candidates_df['tag_score'] = 0.0
    elif has_tags:
        print("[OK] Retrieved {} candidates".format(len(candidates_df)))
        print("     Tag score range: [{:.3f} - {:.3f}]".format(
            candidates_df['tag_score'].min(),
            candidates_df['tag_score'].max()
        ))
    else:
        print("[ERROR] No candidates found. Aborting.")
        return pd.DataFrame()

    # PHASE 7: AUDIO SCORING
    print("\n[PHASE 7] AUDIO SCORING")
    print("-" * 40)

    if criteria_audio and not candidates_df.empty:
        audio_scores = calculate_audio_match(candidates_df, criteria_audio)
        candidates_df['audio_score'] = audio_scores
        print("[OK] Computed audio scores for {} candidates".format(len(candidates_df)))
        print("     Audio score range: [{:.3f} - {:.3f}]".format(
            candidates_df['audio_score'].min(),
            candidates_df['audio_score'].max()
        ))
    else:
        candidates_df['audio_score'] = 1.0
        print("[INFO] No audio criteria - neutral score applied")

    # PHASE 8: SCORE FUSION
    print("\n[PHASE 8] SCORE FUSION")
    print("-" * 40)

    audio_weight = SCORING_CONFIG['audio_weight'] if has_tags else 1.0
    print("     Audio weight: {:.0%}, Tag weight: {:.0%}".format(audio_weight, 1 - audio_weight))

    candidates_merged = merge_tag_and_audio_scores(
        candidates_df,
        audio_weight=audio_weight,
        use_tags=has_tags
    )

    print("[OK] Final score range: [{:.3f} - {:.3f}]".format(
        candidates_merged['score'].min(),
        candidates_merged['score'].max()
    ))

    # PHASE 9: TIERING
    print("\n[PHASE 9] TIERING")
    print("-" * 40)

    t_high, t_mid = calculate_dynamic_thresholds(
        candidates_merged,
        high_threshold=WORKSET_CONFIG['min_absolute_high'],
        mid_threshold=WORKSET_CONFIG['min_absolute_mid']
    )

    tier_a, tier_b, tier_c = tier_by_score(candidates_merged, t_high, t_mid)

    print("     Thresholds: t_high={:.3f}, t_mid={:.3f}".format(t_high, t_mid))
    print("     Tier A (Premium):   {:>5} tracks".format(len(tier_a)))
    print("     Tier B (Standard):  {:>5} tracks".format(len(tier_b)))
    print("     Tier C (Low):       {:>5} tracks".format(len(tier_c)))

    # PHASE 10: WORKING SET CONSTRUCTION
    print("\n[PHASE 10] WORKING SET CONSTRUCTION")
    print("-" * 40)

    working = build_working_set(
        tier_a, tier_b, tier_c,
        target_pool_size=WORKSET_CONFIG['target_pool_size'],
        min_required_size=WORKSET_CONFIG['min_required_size'],
        popularity_rescue_ratio=WORKSET_CONFIG['popularity_rescue_ratio']
    )

    if working.empty:
        print("[ERROR] Empty working set. Aborting.")
        return pd.DataFrame()

    pop_high, pop_mid, pop_low = bucket_by_popularity(
        working,
        POPULARITY_CONFIG['p_high'],
        POPULARITY_CONFIG['p_mid']
    )

    print("[OK] Working set: {} tracks".format(len(working)))
    print("     Popularity buckets: High={}, Mid={}, Low={}".format(
        len(pop_high), len(pop_mid), len(pop_low)
    ))

    # PHASE 11: FINAL SAMPLING
    print("\n[PHASE 11] FINAL SAMPLING")
    print("-" * 40)
    print("     Target: {} tracks, Alpha: {}, Shuffle: {}".format(
        n_songs,
        sampling_config['alpha'],
        sampling_config['shuffle']
    ))

    final_playlist = sample_final_songs(
        working,
        POPULARITY_CONFIG,
        sampling_config
    )

    # OUTPUT
    print("\n" + "=" * 80)
    print("COMPLETE: {} tracks generated".format(len(final_playlist)))
    print("=" * 80)

    display_cols = ["name", "artist", "popularity", "score", "tag_score", "audio_score"]

    if criteria_audio:
        audio_feature_names = [feat for feat, _ in criteria_audio]
        for col in audio_feature_names[:3]:
            if col in final_playlist.columns:
                display_cols.append(col)

    display_cols = [col for col in display_cols if col in final_playlist.columns]

    display_df = final_playlist[display_cols].copy()
    for col in display_df.columns:
        if display_df[col].dtype in ['float64', 'float32']:
            display_df[col] = display_df[col].round(3)

    print("\n" + display_df.to_string(index=True))
    print("-" * 80)

    return final_playlist

In [ ]:
# UŻYCIE
playlist = generate_playlist("fajna szybka muzyka rockowa", n_songs=15)

PLAYLIST GENERATION PIPELINE
  Input prompt: 'fajna szybka muzyka rockowa'
  Requested tracks: 15

[PHASE 1] PHRASE EXTRACTION
----------------------------------------
Prompt: 'fajna szybka muzyka rockowa' 
-> ['fajna', 'rockowa', 'szybka']
[OK] Extracted 3 phrases: ['fajna', 'rockowa', 'szybka']


PHASE 2: CLASSIFICATION & ROUTING

PHRASE                         ROUTE      CATEGORY                 
-----------------------------------------------------------------
fajna                          AUDIO      none                     
rockowa                        TAGS       music_genre              
szybka                         AUDIO      none                     
-----------------------------------------------------------------

ROUTING RESULT:
  TAGS  -> ['rockowa']
  AUDIO -> ['fajna', 'szybka']
  Language: SW

[PHASE 3] AUDIO FEATURE MAPPING
----------------------------------------

FRAZY: ['fajna', 'szybka']
[MATCH:ACTIVITY] 'fajna' -> category: sleep_relax (score: 0.8209)
[MATCH: